### Import Dependencies

In [353]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib as plt

### Set Formulas

In [354]:
def win_rate_by_record(team_df, team, wins, week):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Entering Wins'] >= wins]
    df = df[df['Week'] == week]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        return f'No data.'
    else:
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

In [355]:
def win_rate_by_pf(team_df, team, pf):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['PF'] >= pf]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        return f'No data.'
    else:
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

In [356]:
def win_rate_by_pa(team_df, team, pa):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['PA'] >= pa]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        return f'No data.'
    else:
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

In [357]:
def win_rate_by_opp(team_df, team, opp):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Opp'] == opp]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        return f'No data.'
    else:
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

In [358]:
def avg_pf_by_team(team_df, team, opp):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Opp'] == opp]
    pf_avg = df['PF'].mean()
    pf_avg_rounddown = pf_avg - (pf_avg % 1)
    return pf_avg_rounddown

In [359]:
def avg_pa_by_team(team_df, team, opp):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Opp'] == opp]
    pa_avg = df['PA'].mean()
    pa_avg_roundup = pa_avg + (1 - (pa_avg % 1))
    return pa_avg_roundup

In [360]:
def total_avg(team_df, team, wins, week, opp, pa, pf):
        
    df = team_df[team_df['Home_Team'] == team]
    percentage_sum = 0
    count = 0

    # win_rate_by_ record
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Entering Wins'] >= wins]
    df = df[df['Week'] == week]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        pass
    else:
        win_pt1 = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        percentage_sum = percentage_sum + win_pt1
        count = count + 1

    # win_rate_by_ opp
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Opp'] == opp]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        pass
    else:
        win_pt2 = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        percentage_sum = percentage_sum + win_pt2
        count = count + 1

    # win_rate_by_ avg pf
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['PF'] >= pf]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        pass
    else:
        win_pt5 = win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))
        percentage_sum = percentage_sum + win_pt5
        count = count + 1

    # win_rate_by_ avg pa
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['PA'] >= pa]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        pass
    else:
        win_pt6 = win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))
        percentage_sum = percentage_sum + win_pt6
        count = count + 1

    total_avg = float("{:.2f}".format(percentage_sum/count))
    return total_avg

In [361]:
def odds_summary(team_df, team, opp, week, wins=0, pf=0, pa=0):    
    print(f'Avg Win Rating - Overall: {total_avg(team_df=team_df, team=team, wins=wins, week=week, opp=opp, pa=pa, pf=pf)}%')
    print(f'-----')
    print(f'Against {opp}, Win Rate: {win_rate_by_opp(team_df=team_df, team=team, opp=opp)}%')
    print(f'Week {week}- Wins by: {wins}, Win Rate: {win_rate_by_record(team_df=team_df, team=team, wins=wins, week=week)}%')
    print(f'Avg PF: {avg_pf_by_team(team_df=team_df, team=team, opp=opp)}, Win Rate: {win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))}%')
    print(f'Avg PA: {avg_pa_by_team(team_df=team_df, team=team, opp=opp)}, Win Rate: {win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))}%')

In [362]:
# Generate our categorical variable list
def cat_columns(df):
    cats = df.dtypes[df.dtypes == "object"].index.tolist()
    return cats

In [363]:
# Get unique values in each column with categorical values
def vcount_summary(df, cat_list):
    for i in cat_list:
        print(f'Column: {i}')
        print(f'Unique Values: {df[i].nunique()}')
        print(df[i].value_counts())
        print('-------------')

In [364]:
# Make function to bucket low values into 'other' type
def other_bucket(df, column, cutoff):
    counts = df[column].value_counts()
    replacements = list(counts[counts < cutoff].index)
    
    for i in replacements:
        df[column] = df[column].replace(i, 'Other')

In [365]:
# Encode categorical columns and merge with primary dataframe
def encode_merge(df, cat_list):
    for i in cat_list:
        encode_df = pd.DataFrame(enc.fit_transform(df[i].values.reshape(-1,1)))
        encode_df.columns = enc.get_feature_names([i])
        df = df.merge(encode_df,left_index=True,right_index=True).drop(i,1)
    return df

In [366]:
def clean_yearlies(df, base_team=''):
    
    # Add a 'base team' column
    df['Base_Team'] = base_team
    
    # Unify the data so all null rows are set to zero
    df['PF'].fillna(0, inplace = True)
    df['PA'].fillna(0, inplace = True)
    df['OPassY'].fillna(0, inplace = True)
    df['ORushY'].fillna(0, inplace = True)
    df['TO_lost'].fillna(0, inplace = True)
    df['DPassY'].fillna(0, inplace = True)
    df['DRushY'].fillna(0, inplace = True)
    df['TO_won'].fillna(0, inplace = True)
    df['OTotYd'].fillna(0, inplace = True)
    df['DTotYd'].fillna(0, inplace = True)

    # Caluculate the cumulative sum of each stat by week, EXCLUDING that week's numbers. Should reflect the
    # team's states going into the game, not coming out of it.
    df['PF Tally'] = df['PF'].cumsum() - df['PF']
    df['PA Tally'] = df['PA'].cumsum() - df['PA']
    df['OPassY Tally'] = df['OPassY'].cumsum() - df['OPassY']
    df['ORushY Tally'] = df['ORushY'].cumsum() - df['ORushY']
    df['TO_lost Tally'] = df['TO_lost'].cumsum() - df['TO_lost']
    df['TO_won Tally'] = df['TO_won'].cumsum() - df['TO_won']
    df['DPassY Tally'] = df['DPassY'].cumsum() - df['DPassY']
    df['DRushY Tally'] = df['DRushY'].cumsum() - df['DRushY']
    df['OTotYd Tally'] = df['OTotYd'].cumsum() - df['OTotYd']
    df['DTotYd Tally'] = df['DTotYd'].cumsum() - df['DTotYd']
    
    # Clean 'Results' column so wins have a value of 1 and losses/ties have a value of 0
    df['Results'] = df['Results'].replace(['L', 'T'], 0)
    df['Results'] = df['Results'].replace('W', 1)

    # Caluculate win tallies going into each game
    df = df.rename(columns={"Entering Wins": "Wins Tally"})
    df['Wins Tally'] = 0
    df['Wins Tally'] = df['Results'].cumsum() - df['Results']
    
    # Clean 'week_after_bye' column so only the 'Bye Week' row and the row after it have a value of 1.
    # 'Bye Week' row will be dropped later.
    df['week_after_bye'] = 0
    df.loc[df['Opp'] == 'Bye Week', 'week_after_bye'] = 1
    df['after_bye'] = df['week_after_bye'].cumsum()
    df['week_after_bye'].values[df['week_after_bye'] >= 2] = 0
    df['week_after_bye'] = df['after_bye'].cumsum() - df['after_bye']
    df['after_bye'].values[df['week_after_bye'] >= 2] = 0
    
    df = df.drop(columns=['week_after_bye', 'Rec', 'Date', 'PF', 'PA', 'OPassY', 'ORushY', 'TO_lost', 'TO_won', 'DPassY', 'DRushY', 'OTotYd', 'DTotYd'])
    
    # Delete 'Bye' weeks and change column name
    df = df[df['Opp'] != 'Bye Week']

    # Clean 'Home' column so home games have a value of 1 and away games have a value of 0
    df['Home'] = df['Home'].replace(['@', 'N'], 0)
    df['Home'].fillna(1, inplace = True)
    
    return df

In [367]:
def yearly_data(df, year, HC2_range=0, QB2_range=0, HC='', HC2='none', OC='', DC='', QB='', QB2='none'):
    df['year'] = year
    df = clean_yearlies(df)
    df['Head_Coach'] = HC
    if HC2 == 'none':
        pass
    else:
        df.loc[df['Week'] == HC2_range, 'Head_Coach'] = HC2
    df['OC'] = OC
    df['DC'] = DC
    df['QB'] = QB
    if QB2 == 'none':
        pass
    else:
        df.loc[df['Week'] == QB2_range, 'Head_Coach'] = QB2
        
    df = df[df['Week'] <= 17]
        
    return df

### San Francisco 49ers Data, 2010-2019

In [368]:
# Data Source: https://www.pro-football-reference.com/teams/sfo/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

sf_df2010 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2010.csv')
sf_df2011 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2011.csv')
sf_df2012 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2012.csv')
sf_df2013 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2013.csv')
sf_df2014 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2014.csv')
sf_df2015 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2015.csv')
sf_df2016 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2016.csv')
sf_df2017 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2017.csv')
sf_df2018 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2018.csv')
sf_df2019 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2019.csv')

sf_df2010 = yearly_data(df=sf_df2010, year=2010, 
                        HC='Mike Singletary', HC2_range=17, HC2='Jim Tomsula', 
                        OC='Jimmy Raye', 
                        DC='Greg Manusky', 
                        QB='Alex Smith', 
                        QB2='David Carr', QB2_range=7)
sf_df2010.loc[sf_df2010['Week'] == range(8,13,1), 'QB'] = "Troy Smith"

sf_df2011 = yearly_data(df=sf_df2011, year=2011, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Alex Smith')
sf_df2012 = yearly_data(df=sf_df2012, year=2012, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio',
                        QB='Alex Smith', 
                        QB2='Colin Kaepernick', QB2_range=range(10,18,1))
sf_df2013 = yearly_data(df=sf_df2013, year=2013, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Colin Kaepernick')
sf_df2014 = yearly_data(df=sf_df2014, year=2014, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Colin Kaepernick')
sf_df2015 = yearly_data(df=sf_df2015, year=2015, 
                        HC='Jim Tomsula', 
                        OC='Geep Chryst', 
                        DC='Eric Mangini', 
                        QB='Colin Kaepernick',
                        QB2='Blaine Gabbert', QB2_range=range(10,18,1))
sf_df2016 = yearly_data(df=sf_df2016, year=2016, 
                        HC='Chip Kelly', 
                        OC='Curtis Modkins', 
                        DC="Jim O'Neil", 
                        QB='Blaine Gabbert',
                        QB2='Colin Kaepernick', QB2_range=(6,18,1))
sf_df2017 = yearly_data(df=sf_df2017, year=2017, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Brian Hoyer',
                        QB2='C.J. Beathard', QB2_range=range(7,13,1))
sf_df2017.loc[sf_df2017['Week'] == range(12,18,1), 'QB'] = "Jimmy Garoppolo"

sf_df2018 = yearly_data(df=sf_df2018, year=2018, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo',
                        QB2='C.J. Beathard', QB2_range=range(3,9,1))
sf_df2018.loc[sf_df2018['Week'] == range(8,18,1), 'QB'] = "Nick Mullens"

sf_df2019 = yearly_data(df=sf_df2019, year=2019, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo')

sf_df = sf_df2019.append(sf_df2018, ignore_index=True)
sf_df = sf_df.append(sf_df2017, ignore_index=True)
sf_df = sf_df.append(sf_df2016, ignore_index=True)
sf_df = sf_df.append(sf_df2015, ignore_index=True)
sf_df = sf_df.append(sf_df2014, ignore_index=True)
sf_df = sf_df.append(sf_df2013, ignore_index=True)
sf_df = sf_df.append(sf_df2012, ignore_index=True)
sf_df = sf_df.append(sf_df2011, ignore_index=True)
sf_df = sf_df.append(sf_df2010, ignore_index=True)

sf_df['Base_Team'] = '49ers'
sf_df = sf_df.sort_values(by=['year', 'Week'], ascending=[False, True])

sf_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Tampa Bay Buccaneers,2019,49ers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
1,2,Sun,1.0,1.0,0.0,Cincinnati Bengals,2019,49ers,31.0,17.0,...,4.0,174.0,121.0,256.0,295.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
2,3,Sun,1.0,2.0,1.0,Pittsburgh Steelers,2019,49ers,72.0,34.0,...,5.0,465.0,146.0,827.0,611.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
3,5,Mon,1.0,3.0,1.0,Cleveland Browns,2019,49ers,96.0,54.0,...,7.0,625.0,225.0,1263.0,850.0,1,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
4,6,Sun,1.0,4.0,0.0,Los Angeles Rams,2019,49ers,127.0,57.0,...,11.0,703.0,327.0,1709.0,1030.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,4.0,0.0,Green Bay Packers,2010,49ers,187.0,225.0,...,14.0,2404.0,1081.0,3523.0,3485.0,0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith
156,14,Sun,1.0,4.0,1.0,Seattle Seahawks,2010,49ers,203.0,259.0,...,14.0,2678.0,1217.0,3792.0,3895.0,0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith
157,15,Thu,0.0,5.0,0.0,San Diego Chargers,2010,49ers,243.0,280.0,...,19.0,2955.0,1301.0,4128.0,4256.0,0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith
158,16,Sun,0.0,5.0,0.0,St. Louis Rams,2010,49ers,250.0,314.0,...,19.0,3221.0,1409.0,4320.0,4630.0,0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith


In [369]:
niners_df = sf_df.copy()
niners_df.to_csv('Resources/indv_team_csvs/niners.csv')

### Rams Data, 2010-2019

In [370]:
# Data Source: https://www.pro-football-reference.com/teams/ram/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

rams_df2010 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2010.csv')
rams_df2011 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2011.csv')
rams_df2012 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2012.csv')
rams_df2013 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2013.csv')
rams_df2014 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2014.csv')
rams_df2015 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2015.csv')
rams_df2016 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2016.csv')
rams_df2017 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2017.csv')
rams_df2018 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2018.csv')
rams_df2019 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2019.csv')

rams_df2010 = yearly_data(df=rams_df2010, year=2010, 
                        HC='Steve Spagnuolo', 
                        OC='Pat Shurmur', 
                        DC='Ken Flajole', 
                        QB='Sam Bradford')

rams_df2011 = yearly_data(df=rams_df2011, year=2011, 
                          HC='Steve Spagnuolo', 
                          OC='Josh McDaniels', 
                          DC='Ken Flajole', 
                          QB='Sam Bradford',
                          QB2='A.J. Feeley', QB2_range=range(7,9,1))
rams_df2011.loc[rams_df2011['Week'] == 12, 'QB'] = "Brandon Lloyd"
rams_df2011.loc[rams_df2011['Week'] == 13, 'QB'] = "A.J. Feeley"
rams_df2011.loc[rams_df2011['Week'] == range(15,18,1), 'QB'] = "Kellen Clemens"

rams_df2012 = yearly_data(df=rams_df2012, year=2012, 
                        HC='Jeff Fisher', 
                        OC='Brian Schottenheimer', 
                        DC='Jeff Fisher',
                        QB='Sam Bradford')

rams_df2013 = yearly_data(df=rams_df2013, year=2013, 
                        HC='Jeff Fisher', 
                        OC='Brian Schottenheimer', 
                        DC='Tim Walton', 
                        QB='Sam Bradford')
rams_df2013.loc[rams_df2013['Week'] == range(8,18,1), 'QB'] = "Kellen Clemens"

rams_df2014 = yearly_data(df=rams_df2014, year=2014, 
                        HC='Jeff Fisher', 
                        OC='Brian Schottenheimer', 
                        DC='Gregg Williams', 
                        QB='Shaun Hill',
                        QB2='Austin Davis', QB2_range=(2,11,1))

rams_df2015 = yearly_data(df=rams_df2015, year=2015, 
                        HC='Jeff Fisher', 
                        OC='Rob Boras', 
                        DC='Gregg Williams', 
                        QB='Nick Foles',
                        QB2='Case Keenum', QB2_range=11)
rams_df2015.loc[rams_df2015['Week'] == range(14,18,1), 'QB'] = "Case Keenum"

rams_df2016 = yearly_data(df=rams_df2016, year=2016, 
                        HC='Jeff Fisher', HC2='John Fassel', HC2_range=range(14,18,1),
                        OC='Rob Boras', 
                        DC="Gregg Williams", 
                        QB='Case Keenum',
                        QB2='Jared Goff', QB2_range=range(11,18,1))

rams_df2017 = yearly_data(df=rams_df2017, year=2017, 
                        HC='Sean McVay', 
                        OC='Matt LaFleur', 
                        DC='Wade Phillips', 
                        QB='Jared Goff',
                        QB2='Sean Mannion', QB2_range=17)

rams_df2018 = yearly_data(df=rams_df2018, year=2018, 
                        HC='Sean McVay', 
                        OC='Sean McVay', 
                        DC='Wade Phillips', 
                        QB='Jared Goff')

rams_df2019 = yearly_data(df=rams_df2019, year=2019, 
                        HC='Sean McVay', 
                        OC='Sean McVay', 
                        DC='Wade Phillips', 
                        QB='Jared Goff')

# Make master dataframe for team
rams_df = rams_df2019.append(rams_df2018, ignore_index=True)
rams_df = rams_df.append(rams_df2017, ignore_index=True)
rams_df = rams_df.append(rams_df2016, ignore_index=True)
rams_df = rams_df.append(rams_df2015, ignore_index=True)
rams_df = rams_df.append(rams_df2014, ignore_index=True)
rams_df = rams_df.append(rams_df2013, ignore_index=True)
rams_df = rams_df.append(rams_df2012, ignore_index=True)
rams_df = rams_df.append(rams_df2011, ignore_index=True)
rams_df = rams_df.append(rams_df2010, ignore_index=True)

rams_df['Base_Team'] = 'Rams'
rams_df = rams_df.sort_values(by=['year', 'Week'], ascending=[False, True])

rams_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Carolina Panthers,2019,Rams,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Sean McVay,Sean McVay,Wade Phillips,Jared Goff
1,2,Sun,1.0,1.0,1.0,New Orleans Saints,2019,Rams,30.0,27.0,...,3.0,216.0,127.0,349.0,343.0,0,Sean McVay,Sean McVay,Wade Phillips,Jared Goff
2,3,Sun,1.0,2.0,0.0,Cleveland Browns,2019,Rams,57.0,36.0,...,4.0,403.0,184.0,729.0,587.0,0,Sean McVay,Sean McVay,Wade Phillips,Jared Goff
3,4,Sun,0.0,3.0,1.0,Tampa Bay Buccaneers,2019,Rams,77.0,49.0,...,5.0,578.0,279.0,1073.0,857.0,0,Sean McVay,Sean McVay,Wade Phillips,Jared Goff
4,5,Thu,0.0,3.0,0.0,Seattle Seahawks,2019,Rams,117.0,104.0,...,6.0,954.0,367.0,1591.0,1321.0,0,Sean McVay,Sean McVay,Wade Phillips,Jared Goff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,5.0,0.0,Arizona Cardinals,2010,Rams,213.0,231.0,...,17.0,2635.0,1137.0,3488.0,3772.0,0,Steve Spagnuolo,Pat Shurmur,Ken Flajole,Sam Bradford
156,14,Sun,0.0,6.0,0.0,New Orleans Saints,2010,Rams,232.0,237.0,...,19.0,2754.0,1242.0,3776.0,3996.0,0,Steve Spagnuolo,Pat Shurmur,Ken Flajole,Sam Bradford
157,15,Sun,0.0,6.0,1.0,Kansas City Chiefs,2010,Rams,245.0,268.0,...,22.0,2967.0,1374.0,4103.0,4341.0,0,Steve Spagnuolo,Pat Shurmur,Ken Flajole,Sam Bradford
158,16,Sun,1.0,6.0,1.0,San Francisco 49ers,2010,Rams,258.0,295.0,...,23.0,3140.0,1584.0,4327.0,4724.0,0,Steve Spagnuolo,Pat Shurmur,Ken Flajole,Sam Bradford


In [371]:
rams_df.to_csv('Resources/indv_team_csvs/rams.csv')

### Chargers Data, 2010-2019

In [372]:
# Data Source: https://www.pro-football-reference.com/teams/sdg/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

chargers_df2010 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2010.csv')
chargers_df2011 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2011.csv')
chargers_df2012 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2012.csv')
chargers_df2013 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2013.csv')
chargers_df2014 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2014.csv')
chargers_df2015 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2015.csv')
chargers_df2016 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2016.csv')
chargers_df2017 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2017.csv')
chargers_df2018 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2018.csv')
chargers_df2019 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2019.csv')

chargers_df2010 = yearly_data(df=chargers_df2010, year=2010, 
                        HC='Norv Turner', 
                        OC='Clarence Shelmon', 
                        DC='Ron Rivera', 
                        QB='Philip Rivers')

chargers_df2011 = yearly_data(df=chargers_df2011, year=2011, 
                        HC='Norv Turner', 
                        OC='Clarence Shelmon', 
                        DC='Greg Manusky', 
                        QB='Philip Rivers')

chargers_df2012 = yearly_data(df=chargers_df2012, year=2012, 
                        HC='Norv Turner', 
                        OC='Hal Hunter', 
                        DC='John Pagano',
                        QB='Philip Rivers')

chargers_df2013 = yearly_data(df=chargers_df2013, year=2013, 
                        HC='Mike McCoy', 
                        OC='Ken Whisenhunt', 
                        DC='John Pagano', 
                        QB='Philip Rivers')

chargers_df2014 = yearly_data(df=chargers_df2014, year=2014, 
                        HC='Mike McCoy', 
                        OC='Frank Reich', 
                        DC='John Pagano', 
                        QB='Philip Rivers')

chargers_df2015 = yearly_data(df=chargers_df2015, year=2015, 
                        HC='Mike McCoy', 
                        OC='Frank Reich', 
                        DC='John Pagano', 
                        QB='')

chargers_df2016 = yearly_data(df=chargers_df2016, year=2016, 
                        HC='Mike McCoy', 
                        OC='Ken Whisenhunt', 
                        DC='John Pagano', 
                        QB='Philip Rivers')

chargers_df2017 = yearly_data(df=chargers_df2017, year=2017, 
                        HC='Anthony Lynn', 
                        OC='Ken Whisenhunt', 
                        DC='Gus Bradley', 
                        QB='Philip Rivers')

chargers_df2018 = yearly_data(df=chargers_df2018, year=2018, 
                        HC='Anthony Lynn', 
                        OC='Ken Whisenhunt', 
                        DC='Gus Bradley', 
                        QB='Philip Rivers')

chargers_df2019 = yearly_data(df=chargers_df2019, year=2019, 
                        HC='Anthony Lynn', 
                        OC='Ken Whisenhunt', 
                        DC='Gus Bradley', 
                        QB='Philip Rivers')

# Make master dataframe for team
chargers_df = chargers_df2019.append(chargers_df2018, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2017, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2016, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2015, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2014, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2013, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2012, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2011, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2010, ignore_index=True)

chargers_df['Base_Team'] = 'Chargers'
chargers_df = chargers_df.sort_values(by=['year', 'Week'], ascending=[False, True])

chargers_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Indianapolis Colts,2019,Chargers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Anthony Lynn,Ken Whisenhunt,Gus Bradley,Philip Rivers
1,2,Sun,0.0,1.0,0.0,Detroit Lions,2019,Chargers,30.0,24.0,...,0.0,173.0,203.0,435.0,376.0,0,Anthony Lynn,Ken Whisenhunt,Gus Bradley,Philip Rivers
2,3,Sun,0.0,1.0,1.0,Houston Texans,2019,Chargers,40.0,37.0,...,2.0,418.0,297.0,859.0,715.0,0,Anthony Lynn,Ken Whisenhunt,Gus Bradley,Philip Rivers
3,4,Sun,1.0,1.0,0.0,Miami Dolphins,2019,Chargers,60.0,64.0,...,3.0,755.0,336.0,1225.0,1091.0,0,Anthony Lynn,Ken Whisenhunt,Gus Bradley,Philip Rivers
4,5,Sun,0.0,2.0,1.0,Denver Broncos,2019,Chargers,90.0,74.0,...,4.0,916.0,408.0,1615.0,1324.0,0,Anthony Lynn,Ken Whisenhunt,Gus Bradley,Philip Rivers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,6.0,1.0,Oakland Raiders,2010,Chargers,310.0,225.0,...,18.0,2118.0,892.0,4483.0,3010.0,0,Norv Turner,Clarence Shelmon,Ron Rivera,Philip Rivers
156,14,Sun,1.0,6.0,1.0,Kansas City Chiefs,2010,Chargers,323.0,253.0,...,19.0,2235.0,1143.0,4769.0,3378.0,0,Norv Turner,Clarence Shelmon,Ron Rivera,Philip Rivers
157,15,Thu,1.0,7.0,1.0,San Francisco 49ers,2010,Chargers,354.0,253.0,...,19.0,2254.0,1191.0,5195.0,3445.0,0,Norv Turner,Clarence Shelmon,Ron Rivera,Philip Rivers
158,16,Sun,0.0,8.0,0.0,Cincinnati Bengals,2010,Chargers,388.0,260.0,...,20.0,2385.0,1252.0,5569.0,3637.0,0,Norv Turner,Clarence Shelmon,Ron Rivera,Philip Rivers


In [373]:
chargers_df.to_csv('Resources/indv_team_csvs/chargers.csv')

### Seahawks Data, 2010-2019

In [374]:
# Data Source: https://www.pro-football-reference.com/teams/sea/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

seahawks_df2010 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2010.csv')
seahawks_df2011 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2011.csv')
seahawks_df2012 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2012.csv')
seahawks_df2013 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2013.csv')
seahawks_df2014 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2014.csv')
seahawks_df2015 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2015.csv')
seahawks_df2016 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2016.csv')
seahawks_df2017 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2017.csv')
seahawks_df2018 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2018.csv')
seahawks_df2019 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2019.csv')

seahawks_df2010 = yearly_data(df=seahawks_df2010, year=2010, 
                        HC='Pete Carroll', 
                        OC='Jeremy Bates', 
                        DC='Gus Bradley', 
                        QB='Matt Hasselbeck', 
                        QB2='Charlie Whitehurst', QB2_range=range(9,18,8))

seahawks_df2011 = yearly_data(df=seahawks_df2011, year=2011, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Gus Bradley', 
                        QB='Tarvaris Jackson', 
                        QB2='Charlie Whitehurst', QB2_range=range(7,9,1))

seahawks_df2012 = yearly_data(df=seahawks_df2012, year=2012, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Gus Bradley',
                        QB='Russell Wilson')

seahawks_df2013 = yearly_data(df=seahawks_df2013, year=2013, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Dan Quinn', 
                        QB='Russell Wilson')

seahawks_df2014 = yearly_data(df=seahawks_df2014, year=2014, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Dan Quinn', 
                        QB='Russell Wilson')

seahawks_df2015 = yearly_data(df=seahawks_df2015, year=2015, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Kris Richard', 
                        QB='Russell Wilson')

seahawks_df2016 = yearly_data(df=seahawks_df2016, year=2016, 
                        HC='Pete Carroll',
                        OC='Darrell Bevell', 
                        DC='Kris Richard', 
                        QB='Russell Wilson')

seahawks_df2017 = yearly_data(df=seahawks_df2017, year=2017, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Kris Richard', 
                        QB='Russell Wilson')

seahawks_df2018 = yearly_data(df=seahawks_df2018, year=2018, 
                        HC='Pete Carroll', 
                        OC='Brian Schottenheimer', 
                        DC='Ken Norton', 
                        QB='Russell Wilson')

seahawks_df2019 = yearly_data(df=seahawks_df2019, year=2019, 
                        HC='Pete Carroll', 
                        OC='Brian Schottenheimer', 
                        DC='Ken Norton', 
                        QB='Russell Wilson')

# Make master dataframe for team
seahawks_df = seahawks_df2019.append(seahawks_df2018, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2017, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2016, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2015, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2014, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2013, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2012, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2011, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2010, ignore_index=True)

seahawks_df['Base_Team'] = 'Seahawks'
seahawks_df = seahawks_df.sort_values(by=['year', 'Week'], ascending=[False, True])

seahawks_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Cincinnati Bengals,2019,Seahawks,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
1,2,Sun,1.0,1.0,0.0,Pittsburgh Steelers,2019,Seahawks,21.0,20.0,...,3.0,395.0,34.0,232.0,429.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
2,3,Sun,0.0,2.0,1.0,New Orleans Saints,2019,Seahawks,49.0,46.0,...,4.0,575.0,115.0,657.0,690.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
3,4,Sun,1.0,2.0,0.0,Arizona Cardinals,2019,Seahawks,76.0,79.0,...,5.0,752.0,203.0,1171.0,955.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
4,5,Thu,1.0,3.0,1.0,Los Angeles Rams,2019,Seahawks,103.0,89.0,...,6.0,958.0,318.0,1511.0,1276.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,5.0,1.0,Carolina Panthers,2010,Seahawks,209.0,275.0,...,19.0,3065.0,1324.0,3292.0,4389.0,0,Pete Carroll,Jeremy Bates,Gus Bradley,Matt Hasselbeck
156,14,Sun,0.0,6.0,0.0,San Francisco 49ers,2010,Seahawks,240.0,289.0,...,20.0,3217.0,1455.0,3663.0,4672.0,0,Pete Carroll,Jeremy Bates,Gus Bradley,Matt Hasselbeck
157,15,Sun,0.0,6.0,1.0,Atlanta Falcons,2010,Seahawks,261.0,329.0,...,20.0,3458.0,1550.0,4024.0,5008.0,0,Pete Carroll,Jeremy Bates,Gus Bradley,Matt Hasselbeck
158,16,Sun,0.0,6.0,0.0,Tampa Bay Buccaneers,2010,Seahawks,279.0,363.0,...,21.0,3626.0,1648.0,4258.0,5274.0,0,Pete Carroll,Jeremy Bates,Gus Bradley,Matt Hasselbeck


In [375]:
seahawks_df.to_csv('Resources/indv_team_csvs/seahawks.csv')

### Cardinals Data, 2010-2019

In [376]:
# Data Source: https://www.pro-football-reference.com/teams/crd/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

cardinals_df2010 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2010.csv')
cardinals_df2011 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2011.csv')
cardinals_df2012 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2012.csv')
cardinals_df2013 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2013.csv')
cardinals_df2014 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2014.csv')
cardinals_df2015 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2015.csv')
cardinals_df2016 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2016.csv')
cardinals_df2017 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2017.csv')
cardinals_df2018 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2018.csv')
cardinals_df2019 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2019.csv')

cardinals_df2010 = yearly_data(df=cardinals_df2010, year=2010, 
                        HC='Ken Whisenhunt', 
                        OC='Mike Miller', 
                        DC='Ray Horton', 
                        QB='Derek Anderson', 
                        QB2='John Skelton', QB2_range=range(14,18,1))
cardinals_df2010.loc[cardinals_df2010['Week'] == range(5,9,1), 'QB'] = "Max Hall"

cardinals_df2011 = yearly_data(df=cardinals_df2011, year=2011, 
                        HC='Ken Whisenhunt', 
                        OC='Mike Miller', 
                        DC='Ray Horton', 
                        QB='Kevin Kolb', 
                        QB2='John Skelton', QB2_range=range(9,12,1))
cardinals_df2011.loc[cardinals_df2011['Week'] == 17, 'QB'] = "John Skelton"

cardinals_df2012 = yearly_data(df=cardinals_df2012, year=2012, 
                        HC='Ken Whisenhunt', 
                        OC='Mike Miller', 
                        DC='Ray Horton',
                        QB='John Skelton', 
                        QB2='Kevin Kolb', QB2_range=range(2,7,1))
cardinals_df2012.loc[cardinals_df2012['Week'] == range(12,14,1), 'QB'] = "Ryan Lindley"
cardinals_df2012.loc[cardinals_df2012['Week'] == range(15,16,1), 'QB'] = "Ryan Lindley"
cardinals_df2012.loc[cardinals_df2012['Week'] == 17, 'QB'] = "Bryan Hoyer"

cardinals_df2013 = yearly_data(df=cardinals_df2013, year=2013, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='Todd Bowles', 
                        QB='Carson Palmer')

cardinals_df2014 = yearly_data(df=cardinals_df2014, year=2014, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='Todd Bowles', 
                        QB='Drew Stanton', 
                        QB2='Carson Palmer', QB2_range=1)
cardinals_df2014.loc[cardinals_df2014['Week'] == range(6,11,1), 'QB'] = "Carson Palmer"
cardinals_df2014.loc[cardinals_df2014['Week'] == range(16,18,1), 'QB'] = "Ryan Lindley"

cardinals_df2015 = yearly_data(df=cardinals_df2015, year=2015, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='James Bettcher', 
                        QB='Carson Palmer')

cardinals_df2016 = yearly_data(df=cardinals_df2016, year=2016, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='James Bettcher', 
                        QB='Carson Palmer', 
                        QB2='Drew Stanton', QB2_range=5)

cardinals_df2017 = yearly_data(df=cardinals_df2017, year=2017, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='James Bettcher', 
                        QB='Carson Palmer',
                        QB2="Drew Stanton", QB2_range=range(9,18,1))
cardinals_df2017.loc[cardinals_df2017['Week'] == range(10,16,1), 'QB'] = 'Blaine Gabbert'

cardinals_df2018 = yearly_data(df=cardinals_df2018, year=2018, 
                        HC='Steve Wilks', 
                        OC='Mike McCoy', 
                        DC='Al Holcomb', 
                        QB='Josh Rosen', 
                        QB2='Sam Bradford', QB2_range=range(1,4,1))

cardinals_df2019 = yearly_data(df=cardinals_df2019, year=2019, 
                        HC='Kliff Kingsbury', 
                        OC='Kliff Kingsbury', 
                        DC='Vance Joseph', 
                        QB='Kyler Murray')

# Make master dataframe for team
cardinals_df = cardinals_df2019.append(cardinals_df2018, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2017, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2016, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2015, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2014, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2013, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2012, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2011, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2010, ignore_index=True)

cardinals_df['Base_Team'] = 'Cardinals'
cardinals_df = cardinals_df.sort_values(by=['year', 'Week'], ascending=[False, True])

cardinals_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Detroit Lions,2019,Cardinals,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
1,2,Sun,0.0,0.0,0.0,Baltimore Ravens,2019,Cardinals,27.0,27.0,...,2.0,361.0,116.0,387.0,477.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
2,3,Sun,0.0,0.0,1.0,Carolina Panthers,2019,Cardinals,44.0,50.0,...,2.0,619.0,298.0,736.0,917.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
3,4,Sun,0.0,0.0,1.0,Seattle Seahawks,2019,Cardinals,64.0,88.0,...,3.0,859.0,471.0,984.0,1330.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
4,5,Sun,1.0,0.0,0.0,Cincinnati Bengals,2019,Cardinals,74.0,115.0,...,3.0,1084.0,586.0,1305.0,1670.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,4.0,0.0,New York Jets,2010,Cardinals,180.0,227.0,...,23.0,2241.0,1366.0,3212.0,3607.0,0,Ken Whisenhunt,Mike Miller,Ray Horton,Derek Anderson
156,14,Sun,0.0,4.0,0.0,Seattle Seahawks,2010,Cardinals,186.0,234.0,...,27.0,2353.0,1543.0,3349.0,3896.0,0,Ken Whisenhunt,Mike Miller,Ray Horton,Derek Anderson
157,15,Sun,1.0,4.0,1.0,Detroit Lions,2010,Cardinals,186.0,292.0,...,28.0,2562.0,1827.0,3503.0,4389.0,0,Ken Whisenhunt,Mike Miller,Ray Horton,Derek Anderson
158,16,Sun,0.0,5.0,1.0,Chicago Bears,2010,Cardinals,224.0,302.0,...,32.0,2790.0,1911.0,3699.0,4701.0,0,Ken Whisenhunt,Mike Miller,Ray Horton,Derek Anderson


In [377]:
cardinals_df.to_csv('Resources/indv_team_csvs/cardinals.csv')

### Packers Data, 2010-2019

In [378]:
# Data Source: https://www.pro-football-reference.com/teams/gnb/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

packers_df2010 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2010.csv')
packers_df2011 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2011.csv')
packers_df2012 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2012.csv')
packers_df2013 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2013.csv')
packers_df2014 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2014.csv')
packers_df2015 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2015.csv')
packers_df2016 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2016.csv')
packers_df2017 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2017.csv')
packers_df2018 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2018.csv')
packers_df2019 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2019.csv')

packers_df2010 = yearly_data(df=packers_df2010, year=2010, 
                        HC='Mike McCarthy', 
                        OC='Joe Philbin', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers', 
                        QB2='Matt Flynn', QB2_range=15)

packers_df2011 = yearly_data(df=packers_df2011, year=2011, 
                        HC='Mike McCarthy', 
                        OC='Joe Philbin', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers', 
                        QB2='Matt Flynn', QB2_range=17)

packers_df2012 = yearly_data(df=packers_df2012, year=2012, 
                        HC='Mike McCarthy', 
                        OC='Tom Clements', 
                        DC='Dom Capers',
                        QB='Aaron Rodgers')

packers_df2013 = yearly_data(df=packers_df2013, year=2013, 
                        HC='Mike McCarthy', 
                        OC='Tom Clements', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers', 
                        QB2='Matt Flynn', QB2_range=range(13,17,1))
packers_df2013.loc[packers_df2013['Week'] == range(11,13,1), 'QB'] = "Scott Tolzien"
packers_df2013.loc[packers_df2013['Week'] == 10, 'QB'] = "Seneca Wallace"

packers_df2014 = yearly_data(df=packers_df2014, year=2014, 
                        HC='Mike McCarthy', 
                        OC='Tom Clements', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers')

packers_df2015 = yearly_data(df=packers_df2015, year=2015, 
                        HC='Mike McCarthy', 
                        OC='Edgar Bennett', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers')

packers_df2016 = yearly_data(df=packers_df2016, year=2016, 
                        HC='Mike McCarthy',
                        OC='Edgar Bennett', 
                        DC="Dom Capers", 
                        QB='Aaron Rodgers')

packers_df2017 = yearly_data(df=packers_df2017, year=2017, 
                        HC='Mike McCarthy', 
                        OC='Edgar Bennett', 
                        DC="Dom Capers", 
                        QB='Brett Hundley', 
                        QB2='Aaron Rodgers', QB2_range=range(1,7,1))
packers_df2017.loc[packers_df2017['Week'] == 15, 'QB'] = "Aaron Rodgers"

packers_df2018 = yearly_data(df=packers_df2018, year=2018, 
                        HC='Mike McCarthy', HC2='Joe Philbin', HC2_range=range(14,18,1), 
                        OC='Joe Philbin', 
                        DC='Mike Pettine', 
                        QB='Aaron Rodgers')

packers_df2019 = yearly_data(df=packers_df2019, year=2019, 
                        HC='Matt LaFleur', 
                        OC='Nathaniel Hackett', 
                        DC='Mike Pettine', 
                        QB='Aaron Rodgers')

# Make master dataframe for team
packers_df = packers_df2019.append(packers_df2018, ignore_index=True)
packers_df = packers_df.append(packers_df2017, ignore_index=True)
packers_df = packers_df.append(packers_df2016, ignore_index=True)
packers_df = packers_df.append(packers_df2015, ignore_index=True)
packers_df = packers_df.append(packers_df2014, ignore_index=True)
packers_df = packers_df.append(packers_df2013, ignore_index=True)
packers_df = packers_df.append(packers_df2012, ignore_index=True)
packers_df = packers_df.append(packers_df2011, ignore_index=True)
packers_df = packers_df.append(packers_df2010, ignore_index=True)

packers_df['Base_Team'] = 'Packers'
packers_df = packers_df.sort_values(by=['year', 'Week'], ascending=[False, True])

packers_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Thu,1.0,0.0,0.0,Chicago Bears,2019,Packers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
1,2,Sun,1.0,1.0,1.0,Minnesota Vikings,2019,Packers,10.0,3.0,...,1.0,208.0,46.0,213.0,254.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
2,3,Sun,1.0,2.0,1.0,Denver Broncos,2019,Packers,31.0,19.0,...,5.0,431.0,244.0,548.0,675.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
3,4,Thu,0.0,3.0,1.0,Philadelphia Eagles,2019,Packers,58.0,35.0,...,8.0,592.0,393.0,860.0,985.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
4,5,Sun,1.0,3.0,0.0,Dallas Cowboys,2019,Packers,85.0,69.0,...,8.0,752.0,569.0,1351.0,1321.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,7.0,1.0,San Francisco 49ers,2010,Packers,269.0,166.0,...,21.0,2290.0,1238.0,3894.0,3528.0,0,Mike McCarthy,Joe Philbin,Dom Capers,Aaron Rodgers
156,14,Sun,0.0,8.0,0.0,Detroit Lions,2010,Packers,303.0,182.0,...,22.0,2462.0,1335.0,4304.0,3797.0,0,Mike McCarthy,Joe Philbin,Dom Capers,Aaron Rodgers
157,15,Sun,0.0,8.0,0.0,New England Patriots,2010,Packers,306.0,189.0,...,24.0,2558.0,1525.0,4562.0,4083.0,0,Matt Flynn,Joe Philbin,Dom Capers,Aaron Rodgers
158,16,Sun,1.0,8.0,1.0,New York Giants,2010,Packers,333.0,220.0,...,24.0,2694.0,1638.0,4931.0,4332.0,0,Mike McCarthy,Joe Philbin,Dom Capers,Aaron Rodgers


In [379]:
packers_df.to_csv('Resources/indv_team_csvs/packers.csv')

### Saints Data, 2010-2019

In [380]:
# Data Source: https://www.pro-football-reference.com/teams/nor/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

saints_df2010 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2010.csv')
saints_df2011 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2011.csv')
saints_df2012 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2012.csv')
saints_df2013 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2013.csv')
saints_df2014 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2014.csv')
saints_df2015 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2015.csv')
saints_df2016 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2016.csv')
saints_df2017 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2017.csv')
saints_df2018 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2018.csv')
saints_df2019 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2019.csv')

saints_df2010 = yearly_data(df=saints_df2010, year=2010, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Gregg Williams', 
                        QB='Drew Brees')

saints_df2011 = yearly_data(df=saints_df2011, year=2011, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Gregg Williams', 
                        QB='Drew Brees')

saints_df2012 = yearly_data(df=saints_df2012, year=2012, 
                        HC='Aaron Kromer', HC2='Joe Vitt', HC2_range=range(8,18,1), 
                        OC='Pete Carmichael', 
                        DC='Steve Spagnuolo',
                        QB='Drew Brees')

saints_df2013 = yearly_data(df=saints_df2013, year=2013, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Rob Ryan', 
                        QB='Drew Brees')

saints_df2014 = yearly_data(df=saints_df2014, year=2014, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Rob Ryan', 
                        QB='Drew Brees')

saints_df2015 = yearly_data(df=saints_df2015, year=2015, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Rob Ryan', 
                        QB='Drew Brees', 
                        QB2='Luke McCown', QB2_range=3)
saints_df2015.loc[saints_df2015['Week'] == range(12,18,1), 'DC'] = "Dennis Allen"

saints_df2016 = yearly_data(df=saints_df2016, year=2016, 
                        HC='Sean Payton',
                        OC='Pete Carmichael', 
                        DC="Dennis Allen", 
                        QB='Drew Brees')

saints_df2017 = yearly_data(df=saints_df2017, year=2017, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC="Dennis Allen", 
                        QB='Drew Brees')

saints_df2018 = yearly_data(df=saints_df2018, year=2018, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC="Dennis Allen", 
                        QB='Drew Brees', 
                        QB2='Teddy Bridgewater', QB2_range=17)

saints_df2019 = yearly_data(df=saints_df2019, year=2019, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC="Dennis Allen", 
                        QB='Drew Brees', 
                        QB2='Teddy Bridgewater', QB2_range=range(3,8,1))

# Make master dataframe for team
saints_df = saints_df2019.append(saints_df2018, ignore_index=True)
saints_df = saints_df.append(saints_df2017, ignore_index=True)
saints_df = saints_df.append(saints_df2016, ignore_index=True)
saints_df = saints_df.append(saints_df2015, ignore_index=True)
saints_df = saints_df.append(saints_df2014, ignore_index=True)
saints_df = saints_df.append(saints_df2013, ignore_index=True)
saints_df = saints_df.append(saints_df2012, ignore_index=True)
saints_df = saints_df.append(saints_df2011, ignore_index=True)
saints_df = saints_df.append(saints_df2010, ignore_index=True)

saints_df['Base_Team'] = 'Saints'
saints_df = saints_df.sort_values(by=['year', 'Week'], ascending=[False, True])

saints_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Mon,1.0,0.0,1.0,Houston Texans,2019,Saints,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
1,2,Sun,0.0,1.0,0.0,Los Angeles Rams,2019,Saints,30.0,28.0,...,1.0,234.0,180.0,510.0,414.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
2,3,Sun,1.0,1.0,0.0,Seattle Seahawks,2019,Saints,39.0,55.0,...,2.0,499.0,295.0,754.0,794.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
3,4,Sun,1.0,2.0,1.0,Dallas Cowboys,2019,Saints,72.0,82.0,...,3.0,905.0,403.0,1019.0,1308.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
4,5,Sun,1.0,3.0,1.0,Tampa Bay Buccaneers,2019,Saints,84.0,92.0,...,6.0,1117.0,448.0,1285.0,1565.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,8.0,0.0,Cincinnati Bengals,2010,Saints,265.0,197.0,...,18.0,2176.0,1198.0,4237.0,3374.0,0,Sean Payton,Pete Carmichael,Gregg Williams,Drew Brees
156,14,Sun,1.0,9.0,1.0,St. Louis Rams,2010,Saints,299.0,227.0,...,18.0,2391.0,1294.0,4673.0,3685.0,0,Sean Payton,Pete Carmichael,Gregg Williams,Drew Brees
157,15,Sun,0.0,10.0,0.0,Baltimore Ravens,2010,Saints,330.0,240.0,...,21.0,2582.0,1430.0,5018.0,4012.0,0,Sean Payton,Pete Carmichael,Gregg Williams,Drew Brees
158,16,Mon,1.0,10.0,0.0,Atlanta Falcons,2010,Saints,354.0,270.0,...,21.0,2730.0,1638.0,5287.0,4368.0,0,Sean Payton,Pete Carmichael,Gregg Williams,Drew Brees


In [381]:
saints_df.to_csv('Resources/indv_team_csvs/saints.csv')

### Giants Data, 2010-2019

In [382]:
# Data Source: https://www.pro-football-reference.com/teams/nyg/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

giants_df2010 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2010.csv')
giants_df2011 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2011.csv')
giants_df2012 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2012.csv')
giants_df2013 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2013.csv')
giants_df2014 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2014.csv')
giants_df2015 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2015.csv')
giants_df2016 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2016.csv')
giants_df2017 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2017.csv')
giants_df2018 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2018.csv')
giants_df2019 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2019.csv')

giants_df2010 = yearly_data(df=giants_df2010, year=2010, 
                        HC='Tom Coughlin', 
                        OC='Kevin Gilbride', 
                        DC='Perry Fewell', 
                        QB='Eli Manning')

giants_df2011 = yearly_data(df=giants_df2011, year=2011, 
                        HC='Tom Coughlin', 
                        OC='Kevin Gilbride', 
                        DC='Perry Fewell', 
                        QB='Eli Manning')

giants_df2012 = yearly_data(df=giants_df2012, year=2012, 
                        HC='Tom Coughlin', 
                        OC='Kevin Gilbride', 
                        DC='Perry Fewell',
                        QB='Eli Manning')

giants_df2013 = yearly_data(df=giants_df2013, year=2013, 
                        HC='Tom Coughlin', 
                        OC='Kevin Gilbride', 
                        DC='Perry Fewell', 
                        QB='Eli Manning')

giants_df2014 = yearly_data(df=giants_df2014, year=2014, 
                        HC='Tom Coughlin', 
                        OC='Ben McAdoo', 
                        DC='Perry Fewell', 
                        QB='Eli Manning')

giants_df2015 = yearly_data(df=giants_df2015, year=2015, 
                        HC='Tom Coughlin', 
                        OC='Ben McAdoo', 
                        DC='Steve Spagnuolo', 
                        QB='Eli Manning')

giants_df2016 = yearly_data(df=giants_df2016, year=2016, 
                        HC='Ben McAdoo',
                        OC='Mike Sullivan', 
                        DC="Steve Spagnuolo", 
                        QB='Eli Manning')

giants_df2017 = yearly_data(df=giants_df2017, year=2017, 
                        HC='Ben McAdoo', HC2='Steve Spagnuolo', HC2_range=range(14,18,1), 
                        OC='Mike Sullivan', 
                        DC="Steve Spagnuolo", 
                        QB='Eli Manning', 
                        QB2='Geno Smith', QB2_range=13)

giants_df2018 = yearly_data(df=giants_df2018, year=2018, 
                        HC='Pat Shurmur', 
                        OC='Mike Shula', 
                        DC='James Bettcher', 
                        QB='Eli Manning')

giants_df2019 = yearly_data(df=giants_df2019, year=2019, 
                        HC='Pat Shurmur', 
                        OC='Mike Shula', 
                        DC='James Bettcher', 
                        QB='Daniel Jones', 
                        QB2='Eli Manning', QB2_range=range(1,3,1))
giants_df2019.loc[giants_df2019['Week'] == range(14,16,1), 'QB'] = "Eli Manning"

# Make master dataframe for team
giants_df = giants_df2019.append(giants_df2018, ignore_index=True)
giants_df = giants_df.append(giants_df2017, ignore_index=True)
giants_df = giants_df.append(giants_df2016, ignore_index=True)
giants_df = giants_df.append(giants_df2015, ignore_index=True)
giants_df = giants_df.append(giants_df2014, ignore_index=True)
giants_df = giants_df.append(giants_df2013, ignore_index=True)
giants_df = giants_df.append(giants_df2012, ignore_index=True)
giants_df = giants_df.append(giants_df2011, ignore_index=True)
giants_df = giants_df.append(giants_df2010, ignore_index=True)

giants_df['Base_Team'] = 'Giants'
giants_df = giants_df.sort_values(by=['year', 'Week'], ascending=[False, True])

giants_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Dallas Cowboys,2019,Giants,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Pat Shurmur,Mike Shula,James Bettcher,Daniel Jones
1,2,Sun,0.0,0.0,1.0,Buffalo Bills,2019,Giants,17.0,35.0,...,0.0,405.0,89.0,470.0,494.0,0,Pat Shurmur,Mike Shula,James Bettcher,Daniel Jones
2,3,Sun,1.0,0.0,0.0,Tampa Bay Buccaneers,2019,Giants,31.0,63.0,...,0.0,642.0,240.0,840.0,882.0,0,Pat Shurmur,Mike Shula,James Bettcher,Daniel Jones
3,4,Sun,1.0,1.0,1.0,Washington Redskins,2019,Giants,63.0,94.0,...,1.0,997.0,384.0,1224.0,1381.0,0,Pat Shurmur,Mike Shula,James Bettcher,Daniel Jones
4,5,Sun,0.0,2.0,1.0,Minnesota Vikings,2019,Giants,87.0,97.0,...,5.0,1118.0,439.0,1613.0,1557.0,0,Pat Shurmur,Mike Shula,James Bettcher,Daniel Jones
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,7.0,1.0,Washington Redskins,2010,Giants,277.0,240.0,...,24.0,2050.0,1102.0,4257.0,3152.0,0,Tom Coughlin,Kevin Gilbride,Perry Fewell,Eli Manning
156,14,Mon,1.0,8.0,0.0,Minnesota Vikings,2010,Giants,308.0,247.0,...,30.0,2314.0,1176.0,4615.0,3490.0,0,Tom Coughlin,Kevin Gilbride,Perry Fewell,Eli Manning
157,15,Sun,0.0,9.0,1.0,Philadelphia Eagles,2010,Giants,329.0,250.0,...,31.0,2417.0,1237.0,5010.0,3654.0,0,Tom Coughlin,Kevin Gilbride,Perry Fewell,Eli Manning
158,16,Sun,0.0,9.0,0.0,Green Bay Packers,2010,Giants,360.0,288.0,...,34.0,2638.0,1434.0,5374.0,4072.0,0,Tom Coughlin,Kevin Gilbride,Perry Fewell,Eli Manning


In [383]:
giants_df.to_csv('Resources/indv_team_csvs/giants.csv')

### Bears Data, 2010-2019

In [384]:
# Data Source: https://www.pro-football-reference.com/teams/chi/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

bears_df2010 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2010.csv')
bears_df2011 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2011.csv')
bears_df2012 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2012.csv')
bears_df2013 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2013.csv')
bears_df2014 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2014.csv')
bears_df2015 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2015.csv')
bears_df2016 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2016.csv')
bears_df2017 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2017.csv')
bears_df2018 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2018.csv')
bears_df2019 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2019.csv')

bears_df2010 = yearly_data(df=bears_df2010, year=2010, 
                        HC='Lovie Smith', 
                        OC='Mike Martz', 
                        DC='Rod Marinelli', 
                        QB='Jay Cutler', 
                        QB2='Todd Collins', QB2_range=5)

bears_df2011 = yearly_data(df=bears_df2011, year=2011, 
                        HC='Lovie Smith', 
                        OC='Mike Martz', 
                        DC='Rod Marinelli', 
                        QB='Jay Cutler', 
                        QB2='Caleb Hanie', QB2_range=range(12,16,1))
bears_df2011.loc[bears_df2011['Week'] == range(16,18,1), 'QB'] = "Josh McCown"

bears_df2012 = yearly_data(df=bears_df2012, year=2012, 
                        HC='Lovie Smith', 
                        OC='Mike Tice', 
                        DC='Rod Marinelli',
                        QB='Jay Cutler', 
                        QB2='Jason Campbell', QB2_range=11)

bears_df2013 = yearly_data(df=bears_df2013, year=2013, 
                        HC='Marc Trestman', 
                        OC='Aaron Kromer', 
                        DC='Mel Tucker', 
                        QB='Jay Cutler', 
                        QB2='Josh McCown', QB2_range=9)
bears_df2013.loc[bears_df2013['Week'] == range(11,15,1), 'QB'] = "Josh McCown"

bears_df2014 = yearly_data(df=bears_df2014, year=2014, 
                        HC='Marc Trestman', 
                        OC='Aaron Kromer', 
                        DC='Mel Tucker', 
                        QB='Jay Cutler', 
                        QB2='Jimmy Clausen', QB2_range=16)

bears_df2015 = yearly_data(df=bears_df2015, year=2015, 
                        HC='John Fox', 
                        OC='Adam Gase', 
                        DC='Vic Fangio', 
                        QB='Jay Cutler', 
                        QB2='Jimmy Clausen', QB2_range=3)

bears_df2016 = yearly_data(df=bears_df2016, year=2016, 
                        HC='John Fox',
                        OC='Dowell Loggains', 
                        DC="Vic Fangio", 
                        QB='Jay Cutler', 
                        QB2='Matt Barkley', QB2_range=range(12,18,1))
bears_df2016.loc[bears_df2016['Week'] == range(3,8,1), 'QB'] = "Brian Hoyer"

bears_df2017 = yearly_data(df=bears_df2017, year=2017, 
                        HC='John Fox',
                        OC='Dowell Loggains', 
                        DC="Vic Fangio", 
                        QB='Mitchell Trubisky', 
                        QB2='Mike Glennon', QB2_range=range(1,5,1))

bears_df2018 = yearly_data(df=bears_df2018, year=2018, 
                        HC='Matt Nagy', 
                        OC='Mark Helfrich', 
                        DC='Vic Fangio', 
                        QB='Mitchell Trubisky', 
                        QB2='Chase Daniel', QB2_range=range(12,14,1))

bears_df2019 = yearly_data(df=bears_df2019, year=2019, 
                        HC='Matt Nagy', 
                        OC='Mark Helfrich', 
                        DC='Chuck Pagano', 
                        QB='Mitchell Trubisky', 
                        QB2='Chase Daniel', QB2_range=5)

# Make master dataframe for team
bears_df = bears_df2019.append(bears_df2018, ignore_index=True)
bears_df = bears_df.append(bears_df2017, ignore_index=True)
bears_df = bears_df.append(bears_df2016, ignore_index=True)
bears_df = bears_df.append(bears_df2015, ignore_index=True)
bears_df = bears_df.append(bears_df2014, ignore_index=True)
bears_df = bears_df.append(bears_df2013, ignore_index=True)
bears_df = bears_df.append(bears_df2012, ignore_index=True)
bears_df = bears_df.append(bears_df2011, ignore_index=True)
bears_df = bears_df.append(bears_df2010, ignore_index=True)

bears_df['Base_Team'] = 'Bears'
bears_df = bears_df.sort_values(by=['year', 'Week'], ascending=[False, True])

bears_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Thu,0.0,0.0,1.0,Green Bay Packers,2019,Bears,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Matt Nagy,Mark Helfrich,Chuck Pagano,Mitchell Trubisky
1,2,Sun,1.0,0.0,0.0,Denver Broncos,2019,Bears,3.0,10.0,...,0.0,166.0,47.0,254.0,213.0,0,Matt Nagy,Mark Helfrich,Chuck Pagano,Mitchell Trubisky
2,3,Mon,1.0,1.0,0.0,Washington Redskins,2019,Bears,19.0,24.0,...,1.0,448.0,137.0,527.0,585.0,0,Matt Nagy,Mark Helfrich,Chuck Pagano,Mitchell Trubisky
3,4,Sun,1.0,2.0,1.0,Minnesota Vikings,2019,Bears,50.0,39.0,...,6.0,735.0,206.0,825.0,941.0,0,Matt Nagy,Mark Helfrich,Chuck Pagano,Mitchell Trubisky
4,5,Sun,0.0,3.0,0.0,Oakland Raiders,2019,Bears,66.0,45.0,...,8.0,917.0,246.0,1094.0,1163.0,0,Chase Daniel,Mark Helfrich,Chuck Pagano,Mitchell Trubisky
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,8.0,0.0,Detroit Lions,2010,Bears,222.0,172.0,...,26.0,2417.0,885.0,3293.0,3302.0,0,Lovie Smith,Mike Martz,Rod Marinelli,Jay Cutler
156,14,Sun,0.0,9.0,1.0,New England Patriots,2010,Bears,246.0,192.0,...,26.0,2585.0,1019.0,3604.0,3604.0,0,Lovie Smith,Mike Martz,Rod Marinelli,Jay Cutler
157,15,Mon,1.0,9.0,0.0,Minnesota Vikings,2010,Bears,253.0,228.0,...,26.0,2936.0,1143.0,3789.0,4079.0,0,Lovie Smith,Mike Martz,Rod Marinelli,Jay Cutler
158,16,Sun,1.0,10.0,1.0,New York Jets,2010,Bears,293.0,242.0,...,31.0,3095.0,1257.0,4082.0,4352.0,0,Lovie Smith,Mike Martz,Rod Marinelli,Jay Cutler


In [385]:
bears_df.to_csv('Resources/indv_team_csvs/bears.csv')

### Panthers Data, 2010-2019

In [386]:
# Data Source: https://www.pro-football-reference.com/teams/car/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

panthers_df2010 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2010.csv')
panthers_df2011 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2011.csv')
panthers_df2012 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2012.csv')
panthers_df2013 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2013.csv')
panthers_df2014 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2014.csv')
panthers_df2015 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2015.csv')
panthers_df2016 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2016.csv')
panthers_df2017 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2017.csv')
panthers_df2018 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2018.csv')
panthers_df2019 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2019.csv')

panthers_df2010 = yearly_data(df=panthers_df2010, year=2010, 
                        HC='John Fox', 
                        OC='Jeff Davidson', 
                        DC='Ron Meeks', 
                        QB='Jimmy Clausen', 
                        QB2='Matt Moore', QB2_range=range(1,3,1))
panthers_df2010.loc[panthers_df2010['Week'] == range(7,10,1), 'QB'] = "Matt Moore"
panthers_df2010.loc[panthers_df2010['Week'] == 11, 'QB'] = "Brian St. Pierre"

panthers_df2011 = yearly_data(df=panthers_df2011, year=2011, 
                        HC='Ron Rivera', 
                        OC='Rob Chudzinski', 
                        DC='Sean McDermott', 
                        QB='Cam Newton')

panthers_df2012 = yearly_data(df=panthers_df2012, year=2012, 
                        HC='Ron Rivera', 
                        OC='Rob Chudzinski', 
                        DC='Sean McDermott',
                        QB='Cam Newton')

panthers_df2013 = yearly_data(df=panthers_df2013, year=2013, 
                        HC='Ron Rivera', 
                        OC='Mike Shula', 
                        DC='Sean McDermott', 
                        QB='Cam Newton')

panthers_df2014 = yearly_data(df=panthers_df2014, year=2014, 
                        HC='Ron Rivera', 
                        OC='Mike Shula', 
                        DC='Sean McDermott', 
                        QB='Cam Newton', 
                        QB2='Derek Anderson', QB2_range=1)
panthers_df2014.loc[panthers_df2014['Week'] == 15, 'QB'] = "Derek Anderson"

panthers_df2015 = yearly_data(df=panthers_df2015, year=2015, 
                        HC='Ron Rivera', 
                        OC='Mike Shula', 
                        DC='Sean McDermott', 
                        QB='Cam Newton')

panthers_df2016 = yearly_data(df=panthers_df2016, year=2016, 
                        HC='Ron Rivera',
                        OC='Mike Shula', 
                        DC='Sean McDermott', 
                        QB='Cam Newton', 
                        QB2='Derek Anderson', QB2_range=5)
panthers_df2016.loc[panthers_df2016['Week'] == 13, 'QB'] = "Derek Anderson"

panthers_df2017 = yearly_data(df=panthers_df2017, year=2017, 
                        HC='Ron Rivera', 
                        OC='Mike Shula', 
                        DC='Steve Wilks', 
                        QB='Cam Newton')

panthers_df2018 = yearly_data(df=panthers_df2018, year=2018, 
                        HC='Ron Rivera', 
                        OC='Norv Turner', 
                        DC='Eric Washington', 
                        QB='Cam Newton',
                        QB2='Taylor Heinicke', QB2_range=16)
panthers_df2018.loc[panthers_df2018['Week'] == 17, 'QB'] = "Kyle Allen"

panthers_df2019 = yearly_data(df=panthers_df2019, year=2019, 
                        HC='Ron Rivera', HC2='Perry Fewell', HC2_range=range(14,18,1), 
                        OC='Norv Turner', 
                        DC='Eric Washington', 
                        QB='Kyle Allen',
                        QB2='Cam Newton', QB2_range=range(1,3,1))
panthers_df2019.loc[panthers_df2019['Week'] == 17, 'QB'] = "Will Grier"
panthers_df2019.loc[panthers_df2019['Week'] == range(14,18,1), 'OC'] = "Scott Turner"

# Make master dataframe for team
panthers_df = panthers_df2019.append(panthers_df2018, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2017, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2016, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2015, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2014, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2013, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2012, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2011, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2010, ignore_index=True)

panthers_df['Base_Team'] = 'Panthers'
panthers_df = panthers_df.sort_values(by=['year', 'Week'], ascending=[False, True])

panthers_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Los Angeles Rams,2019,Panthers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Ron Rivera,Norv Turner,Eric Washington,Kyle Allen
1,2,Thu,0.0,0.0,1.0,Tampa Bay Buccaneers,2019,Panthers,27.0,30.0,...,1.0,183.0,166.0,343.0,349.0,0,Ron Rivera,Norv Turner,Eric Washington,Kyle Allen
2,3,Sun,1.0,0.0,0.0,Arizona Cardinals,2019,Panthers,41.0,50.0,...,1.0,372.0,266.0,695.0,638.0,0,Ron Rivera,Norv Turner,Eric Washington,Kyle Allen
3,4,Sun,1.0,1.0,0.0,Houston Texans,2019,Panthers,79.0,70.0,...,3.0,499.0,387.0,1108.0,886.0,0,Ron Rivera,Norv Turner,Eric Washington,Kyle Allen
4,5,Sun,1.0,2.0,1.0,Jacksonville Jaguars,2019,Panthers,95.0,80.0,...,5.0,627.0,523.0,1405.0,1150.0,0,Ron Rivera,Norv Turner,Eric Washington,Kyle Allen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,1.0,0.0,Seattle Seahawks,2010,Panthers,140.0,276.0,...,21.0,2262.0,1436.0,2851.0,3698.0,0,John Fox,Jeff Davidson,Ron Meeks,Jimmy Clausen
156,14,Sun,0.0,1.0,1.0,Atlanta Falcons,2010,Panthers,154.0,307.0,...,23.0,2472.0,1597.0,3134.0,4069.0,0,John Fox,Jeff Davidson,Ron Meeks,Jimmy Clausen
157,15,Sun,1.0,1.0,1.0,Arizona Cardinals,2010,Panthers,164.0,338.0,...,24.0,2672.0,1724.0,3422.0,4396.0,0,John Fox,Jeff Davidson,Ron Meeks,Jimmy Clausen
158,16,Thu,0.0,2.0,0.0,Pittsburgh Steelers,2010,Panthers,183.0,350.0,...,26.0,2847.0,1767.0,3725.0,4614.0,0,John Fox,Jeff Davidson,Ron Meeks,Jimmy Clausen


In [387]:
panthers_df.to_csv('Resources/indv_team_csvs/panthers.csv')

### Bucs Data, 2010-2019

In [388]:
# Data Source: https://www.pro-football-reference.com/teams/tam/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

bucs_df2010 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2010.csv')
bucs_df2011 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2011.csv')
bucs_df2012 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2012.csv')
bucs_df2013 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2013.csv')
bucs_df2014 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2014.csv')
bucs_df2015 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2015.csv')
bucs_df2016 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2016.csv')
bucs_df2017 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2017.csv')
bucs_df2018 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2018.csv')
bucs_df2019 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2019.csv')

bucs_df2010 = yearly_data(df=bucs_df2010, year=2010, 
                        HC='Raheem Morris', 
                        OC='Greg Olson', 
                        DC='Raheem Morris', 
                        QB='Josh Freeman')

bucs_df2011 = yearly_data(df=bucs_df2011, year=2011, 
                        HC='Raheem Morris', 
                        OC='Greg Olson', 
                        DC='Keith Millard', 
                        QB='Josh Freeman', 
                        QB2='Josh Johnson', QB2_range=13)

bucs_df2012 = yearly_data(df=bucs_df2012, year=2012, 
                        HC='Greg Schiano', 
                        OC='Mike Sullivan', 
                        DC='Bill Sheridan',
                        QB='Josh Freeman')

bucs_df2013 = yearly_data(df=bucs_df2013, year=2013, 
                        HC='Greg Schiano', 
                        OC='Mike Sullivan', 
                        DC='Bill Sheridan', 
                        QB='Mike Glennon', 
                        QB2='Josh Freeman', QB2_range=range(1,3,1))

bucs_df2014 = yearly_data(df=bucs_df2014, year=2014, 
                        HC='Lovie Smith', 
                        OC='Jeff Tedford', 
                        DC='Leslie Frazier', 
                        QB='Josh McCown', 
                        QB2='Mike Glennon', QB2_range=range(4,10,1))

bucs_df2015 = yearly_data(df=bucs_df2015, year=2015, 
                        HC='Lovie Smith', 
                        OC='Dirk Koetter', 
                        DC='Leslie Frazier', 
                        QB='Jameis Winston')

bucs_df2016 = yearly_data(df=bucs_df2016, year=2016, 
                        HC='Dirk Koetter',
                        OC='Todd Monken', 
                        DC="Mike Smith", 
                        QB='Jameis Winston')

bucs_df2017 = yearly_data(df=bucs_df2017, year=2017, 
                        HC='Dirk Koetter',
                        OC='Todd Monken', 
                        DC="Mike Smith", 
                        QB='Jameis Winston', 
                        QB2='Ryan Fitzpatrick', QB2_range=range(10,13,1))

bucs_df2018 = yearly_data(df=bucs_df2018, year=2018, 
                        HC='Dirk Koetter',
                        OC='Todd Monken', 
                        DC="Mike Smith", 
                        QB='Jameis Winston', 
                        QB2='Ryan Fitzpatrick', QB2_range=range(1,5,1))
bucs_df2018.loc[bucs_df2018['Week'] == range(9,12,1), 'QB'] = "Ryan Fitzpatrick"
bucs_df2018.loc[bucs_df2018['Week'] == range(7,18,1), 'OC'] = "Mark Duffner"

bucs_df2019 = yearly_data(df=bucs_df2019, year=2019, 
                        HC='Bruce Arians', 
                        OC='Byron Leftwich', 
                        DC='Todd Bowles', 
                        QB='Jameis Winston')

# Make master dataframe for team
bucs_df = bucs_df2019.append(bucs_df2018, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2017, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2016, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2015, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2014, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2013, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2012, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2011, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2010, ignore_index=True)

bucs_df['Base_Team'] = 'Buccaneers'
bucs_df = bucs_df.sort_values(by=['year', 'Week'], ascending=[False, True])

bucs_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,San Francisco 49ers,2019,Buccaneers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Jameis Winston
1,2,Thu,1.0,0.0,0.0,Carolina Panthers,2019,Buccaneers,17.0,31.0,...,2.0,158.0,98.0,295.0,256.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Jameis Winston
2,3,Sun,0.0,1.0,1.0,New York Giants,2019,Buccaneers,37.0,45.0,...,3.0,471.0,137.0,584.0,608.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Jameis Winston
3,4,Sun,1.0,1.0,0.0,Los Angeles Rams,2019,Buccaneers,68.0,77.0,...,5.0,783.0,209.0,1083.0,992.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Jameis Winston
4,5,Sun,0.0,2.0,0.0,New Orleans Saints,2019,Buccaneers,123.0,117.0,...,9.0,1273.0,237.0,1547.0,1510.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Jameis Winston
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,7.0,1.0,Atlanta Falcons,2010,Buccaneers,219.0,223.0,...,22.0,2265.0,1457.0,3513.0,3722.0,0,Raheem Morris,Greg Olson,Raheem Morris,Josh Freeman
156,14,Sun,1.0,7.0,0.0,Washington Redskins,2010,Buccaneers,243.0,251.0,...,24.0,2470.0,1542.0,3838.0,4012.0,0,Raheem Morris,Greg Olson,Raheem Morris,Josh Freeman
157,15,Sun,0.0,8.0,1.0,Detroit Lions,2010,Buccaneers,260.0,267.0,...,25.0,2681.0,1730.0,4203.0,4411.0,0,Raheem Morris,Greg Olson,Raheem Morris,Josh Freeman
158,16,Sun,1.0,8.0,1.0,Seattle Seahawks,2010,Buccaneers,280.0,290.0,...,25.0,2933.0,1911.0,4606.0,4844.0,0,Raheem Morris,Greg Olson,Raheem Morris,Josh Freeman


In [389]:
bucs_df.to_csv('Resources/indv_team_csvs/bucs.csv')

### Falcons Data, 2010-2019

In [390]:
# Data Source: https://www.pro-football-reference.com/teams/atl/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

falcons_df2010 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2010.csv')
falcons_df2011 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2011.csv')
falcons_df2012 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2012.csv')
falcons_df2013 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2013.csv')
falcons_df2014 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2014.csv')
falcons_df2015 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2015.csv')
falcons_df2016 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2016.csv')
falcons_df2017 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2017.csv')
falcons_df2018 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2018.csv')
falcons_df2019 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2019.csv')

falcons_df2010 = yearly_data(df=falcons_df2010, year=2010, 
                        HC='Mike Smith', 
                        OC='Mike Mularkey', 
                        DC='Brian VanGorder', 
                        QB='Matt Ryan')

falcons_df2011 = yearly_data(df=falcons_df2011, year=2011, 
                          HC='Mike Smith', 
                          OC='Mike Mularkey', 
                          DC='Brian VanGorder', 
                          QB='Matt Ryan')

falcons_df2012 = yearly_data(df=falcons_df2012, year=2012, 
                        HC='Mike Smith', 
                        OC='Dirk Koetter', 
                        DC='Mike Nolan',
                        QB='Matt Ryan')

falcons_df2013 = yearly_data(df=falcons_df2013, year=2013, 
                        HC='Mike Smith', 
                        OC='Dirk Koetter', 
                        DC='Mike Nolan', 
                        QB='Matt Ryan')

falcons_df2014 = yearly_data(df=falcons_df2014, year=2014, 
                        HC='Mike Smith', 
                        OC='Dirk Koetter', 
                        DC='Mike Nolan', 
                        QB='Matt Ryan')

falcons_df2015 = yearly_data(df=falcons_df2015, year=2015, 
                        HC='Dan Quinn', 
                        OC='Kyle Shanahan', 
                        DC='Richard Smith', 
                        QB='Matt Ryan')

falcons_df2016 = yearly_data(df=falcons_df2016, year=2016, 
                        HC='Dan Quinn',
                        OC='Kyle Shanahan', 
                        DC="Richard Smith", 
                        QB='Matt Ryan')

falcons_df2017 = yearly_data(df=falcons_df2017, year=2017, 
                        HC='Dan Quinn', 
                        OC='Steve Sarkisian', 
                        DC='Marquand Manuel', 
                        QB='Matt Ryan')

falcons_df2018 = yearly_data(df=falcons_df2018, year=2018, 
                        HC='Dan Quinn', 
                        OC='Steve Sarkisian', 
                        DC='Marquand Manuel', 
                        QB='Matt Ryan')

falcons_df2019 = yearly_data(df=falcons_df2019, year=2019, 
                        HC='Dan Quinn', 
                        OC='Dirk Koetter', 
                        DC='Dan Quinn', 
                        QB='Matt Ryan', 
                        QB2='Matt Schaub', QB2_range=8)

# Make master dataframe for team
falcons_df = falcons_df2019.append(falcons_df2018, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2017, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2016, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2015, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2014, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2013, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2012, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2011, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2010, ignore_index=True)

falcons_df['Base_Team'] = 'Falcons'
falcons_df = falcons_df.sort_values(by=['year', 'Week'], ascending=[False, True])

falcons_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Minnesota Vikings,2019,Falcons,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Dan Quinn,Dirk Koetter,Dan Quinn,Matt Ryan
1,2,Sun,1.0,0.0,1.0,Philadelphia Eagles,2019,Falcons,12.0,28.0,...,0.0,97.0,172.0,345.0,269.0,0,Dan Quinn,Dirk Koetter,Dan Quinn,Matt Ryan
2,3,Sun,0.0,1.0,0.0,Indianapolis Colts,2019,Falcons,36.0,48.0,...,3.0,334.0,221.0,712.0,555.0,0,Dan Quinn,Dirk Koetter,Dan Quinn,Matt Ryan
3,4,Sun,0.0,1.0,1.0,Tennessee Titans,2019,Falcons,60.0,75.0,...,3.0,634.0,300.0,1109.0,934.0,0,Dan Quinn,Dirk Koetter,Dan Quinn,Matt Ryan
4,5,Sun,0.0,1.0,0.0,Houston Texans,2019,Falcons,70.0,99.0,...,3.0,861.0,438.0,1531.0,1299.0,0,Dan Quinn,Dirk Koetter,Dan Quinn,Matt Ryan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,9.0,0.0,Tampa Bay Buccaneers,2010,Falcons,276.0,209.0,...,21.0,2782.0,1031.0,4008.0,3813.0,0,Mike Smith,Mike Mularkey,Brian VanGorder,Matt Ryan
156,14,Sun,1.0,10.0,0.0,Carolina Panthers,2010,Falcons,304.0,233.0,...,22.0,2956.0,1182.0,4298.0,4138.0,0,Mike Smith,Mike Mularkey,Brian VanGorder,Matt Ryan
157,15,Sun,1.0,11.0,0.0,Seattle Seahawks,2010,Falcons,335.0,243.0,...,24.0,3032.0,1394.0,4625.0,4426.0,0,Mike Smith,Mike Mularkey,Brian VanGorder,Matt Ryan
158,16,Mon,0.0,12.0,1.0,New Orleans Saints,2010,Falcons,369.0,261.0,...,27.0,3175.0,1485.0,4891.0,4660.0,0,Mike Smith,Mike Mularkey,Brian VanGorder,Matt Ryan


In [391]:
falcons_df.to_csv('Resources/indv_team_csvs/falcons.csv')

### (13) Washington Data, 2010-2019

In [392]:
# Data Source: https://www.pro-football-reference.com/teams/was/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

wash_df2010 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2010.csv')
wash_df2011 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2011.csv')
wash_df2012 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2012.csv')
wash_df2013 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2013.csv')
wash_df2014 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2014.csv')
wash_df2015 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2015.csv')
wash_df2016 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2016.csv')
wash_df2017 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2017.csv')
wash_df2018 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2018.csv')
wash_df2019 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2019.csv')

wash_df2010 = yearly_data(df=wash_df2010, year=2010, 
                        HC='Mike Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Jim Haslett', 
                        QB='Donovan McNabb', 
                        QB2='Rex Grossman', QB2_range=range(15,18,1))

wash_df2011 = yearly_data(df=wash_df2011, year=2011, 
                        HC='Mike Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Jim Haslett', 
                        QB='Rex Grossman', 
                        QB2='John Beck', QB2_range=range(7,10,1))

wash_df2012 = yearly_data(df=wash_df2012, year=2012, 
                        HC='Mike Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Jim Haslett',
                        QB='Robert Griffin III', 
                        QB2='Kirk Cousins', QB2_range=15)

wash_df2013 = yearly_data(df=wash_df2013, year=2013, 
                        HC='Mike Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Jim Haslett', 
                        QB='Robert Griffin III', 
                        QB2='Kirk Cousins', QB2_range=range(15,18,1))

wash_df2014 = yearly_data(df=wash_df2014, year=2014, 
                        HC='Jay Gruden', 
                        OC='Sean McVay', 
                        DC='Jim Haslett', 
                        QB='Robert Griffin III', 
                        QB2='Kirk Cousins', QB2_range=range(3,8,1))
wash_df2014.loc[wash_df2014['Week'] == 8, 'QB'] = "Colt McCoy"
wash_df2014.loc[wash_df2014['Week'] == range(13,16,1), 'QB'] = "Colt McCoy"

wash_df2015 = yearly_data(df=wash_df2015, year=2015, 
                        HC='Jay Gruden', 
                        OC='Sean McVay', 
                        DC='Joe Barry', 
                        QB='Kirk Cousins')

wash_df2016 = yearly_data(df=wash_df2016, year=2016, 
                        HC='Jay Gruden', 
                        OC='Sean McVay', 
                        DC='Joe Barry', 
                        QB='Kirk Cousins')

wash_df2017 = yearly_data(df=wash_df2017, year=2017, 
                        HC='Jay Gruden', 
                        OC='Matt Cavanaugh', 
                        DC='Greg Manusky', 
                        QB='Kirk Cousins')

wash_df2018 = yearly_data(df=wash_df2018, year=2018, 
                        HC='Jay Gruden', 
                        OC='Matt Cavanaugh', 
                        DC='Greg Manusky', 
                        QB='Alex Smith', 
                        QB2='Colt McCoy', QB2_range=range(12,14,1))
wash_df2018.loc[wash_df2018['Week'] == range(15,18,1), 'QB'] = "Josh Johnson"
wash_df2018.loc[wash_df2018['Week'] == 14, 'QB'] = "Mark Sanchez"

wash_df2019 = yearly_data(df=wash_df2019, year=2019, 
                        HC='Jay Gruden', HC2='Bill Callahan', HC2_range=range(6,18,1), 
                        OC="Kevin O'Connell", 
                        DC='Greg Manusky', 
                        QB='Case Keenum', 
                        QB2='Dwayne Haskins', QB2_range=range(9,17,1))
wash_df2019.loc[wash_df2019['Week'] == 5, 'QB'] = "Colt McCoy"

# Make master dataframe for team
wash_df = wash_df2019.append(wash_df2018, ignore_index=True)
wash_df = wash_df.append(wash_df2017, ignore_index=True)
wash_df = wash_df.append(wash_df2016, ignore_index=True)
wash_df = wash_df.append(wash_df2015, ignore_index=True)
wash_df = wash_df.append(wash_df2014, ignore_index=True)
wash_df = wash_df.append(wash_df2013, ignore_index=True)
wash_df = wash_df.append(wash_df2012, ignore_index=True)
wash_df = wash_df.append(wash_df2011, ignore_index=True)
wash_df = wash_df.append(wash_df2010, ignore_index=True)

wash_df['Base_Team'] = 'Washington'
wash_df = wash_df.sort_values(by=['year', 'Week'], ascending=[False, True])

wash_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Philadelphia Eagles,2019,Washington,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Jay Gruden,Kevin O'Connell,Greg Manusky,Case Keenum
1,2,Sun,0.0,0.0,1.0,Dallas Cowboys,2019,Washington,27.0,32.0,...,0.0,313.0,123.0,398.0,436.0,0,Jay Gruden,Kevin O'Connell,Greg Manusky,Case Keenum
2,3,Mon,0.0,0.0,1.0,Chicago Bears,2019,Washington,48.0,63.0,...,1.0,574.0,336.0,653.0,910.0,0,Jay Gruden,Kevin O'Connell,Greg Manusky,Case Keenum
3,4,Sun,0.0,0.0,0.0,New York Giants,2019,Washington,63.0,94.0,...,2.0,782.0,426.0,1009.0,1208.0,0,Jay Gruden,Kevin O'Connell,Greg Manusky,Case Keenum
4,5,Sun,0.0,0.0,1.0,New England Patriots,2019,Washington,66.0,118.0,...,6.0,1007.0,590.0,1185.0,1597.0,0,Jay Gruden,Kevin O'Connell,Greg Manusky,Colt McCoy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,5.0,0.0,New York Giants,2010,Washington,215.0,262.0,...,21.0,2963.0,1447.0,3660.0,4410.0,0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
156,14,Sun,0.0,5.0,1.0,Tampa Bay Buccaneers,2010,Washington,222.0,293.0,...,22.0,3124.0,1644.0,3998.0,4768.0,0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
157,15,Sun,0.0,5.0,0.0,Dallas Cowboys,2010,Washington,238.0,310.0,...,24.0,3386.0,1747.0,4397.0,5133.0,0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
158,16,Sun,1.0,5.0,0.0,Jacksonville Jaguars,2010,Washington,268.0,343.0,...,24.0,3686.0,1881.0,4738.0,5567.0,0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb


In [393]:
wash_df.to_csv('Resources/indv_team_csvs/wash.csv')

### (14) Ravens Data, 2010-2019

In [394]:
# Data Source: https://www.pro-football-reference.com/teams/rav/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

ravens_df2010 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2010.csv')
ravens_df2011 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2011.csv')
ravens_df2012 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2012.csv')
ravens_df2013 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2013.csv')
ravens_df2014 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2014.csv')
ravens_df2015 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2015.csv')
ravens_df2016 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2016.csv')
ravens_df2017 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2017.csv')
ravens_df2018 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2018.csv')
ravens_df2019 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2019.csv')

ravens_df2010 = yearly_data(df=ravens_df2010, year=2010, 
                        HC='John Harbaugh', 
                        OC='Cam Cameron', 
                        DC='Greg Mattison', 
                        QB='Joe Flacco')

ravens_df2011 = yearly_data(df=ravens_df2011, year=2011, 
                          HC='John Harbaugh', 
                          OC='Cam Cameron', 
                          DC='Chuck Pagano', 
                          QB='Joe Flacco')

ravens_df2012 = yearly_data(df=ravens_df2012, year=2012, 
                        HC='John Harbaugh', 
                        OC='Cam Cameron', 
                        DC='Dean Pees',
                        QB='Joe Flacco')
ravens_df2012.loc[ravens_df2012['Week'] == range(15,18,1), 'OC'] = "Jim Caldwell"

ravens_df2013 = yearly_data(df=ravens_df2013, year=2013, 
                        HC='John Harbaugh', 
                        OC='Jim Caldwell', 
                        DC='Dean Pees', 
                        QB='Joe Flacco')

ravens_df2014 = yearly_data(df=ravens_df2014, year=2014, 
                        HC='John Harbaugh', 
                        OC='Gary Kubiak', 
                        DC='Dean Pees', 
                        QB='Joe Flacco')

ravens_df2015 = yearly_data(df=ravens_df2015, year=2015, 
                        HC='John Harbaugh', 
                        OC='Marc Trestman', 
                        DC='Dean Pees', 
                        QB='Joe Flacco', 
                        QB2='Matt Schaub', QB2_range=range(12,14,1))
ravens_df2015.loc[ravens_df2015['Week'] == range(14,16,1), 'QB'] = "Jimmy Clausen"
ravens_df2015.loc[ravens_df2015['Week'] == range(16,18,1), 'QB'] = "Ryan Mallett"

ravens_df2016 = yearly_data(df=ravens_df2016, year=2016, 
                        HC='John Harbaugh',
                        OC='Marc Trestman', 
                        DC="Dean Pees", 
                        QB='Joe Flacco')
ravens_df2016.loc[ravens_df2016['Week'] == range(6,18,1), 'OC'] = "Marty Mornhinweg"

ravens_df2017 = yearly_data(df=ravens_df2017, year=2017, 
                        HC='John Harbaugh', 
                        OC='Marty Mornhinweg', 
                        DC='Dean Pees', 
                        QB='Joe Flacco')

ravens_df2018 = yearly_data(df=ravens_df2018, year=2018, 
                        HC='John Harbaugh', 
                        OC='Marty Mornhinweg', 
                        DC='Don Martindale', 
                        QB='Joe Flacco', 
                        QB2='Lamar Jackson', QB2_range=range(11,18,1))

ravens_df2019 = yearly_data(df=ravens_df2019, year=2019, 
                        HC='John Harbaugh', 
                        OC='Greg Roman', 
                        DC='Don Martindale', 
                        QB='Lamar Jackson', 
                        QB2='Robert Griffin III', QB2_range=17)

# Make master dataframe for team
ravens_df = ravens_df2019.append(ravens_df2018, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2017, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2016, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2015, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2014, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2013, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2012, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2011, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2010, ignore_index=True)

ravens_df['Base_Team'] = 'Ravens'
ravens_df = ravens_df.sort_values(by=['year', 'Week'], ascending=[False, True])

ravens_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Miami Dolphins,2019,Ravens,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
1,2,Sun,1.0,1.0,1.0,Arizona Cardinals,2019,Ravens,59.0,10.0,...,3.0,179.0,21.0,643.0,200.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
2,3,Sun,0.0,2.0,0.0,Kansas City Chiefs,2019,Ravens,82.0,27.0,...,3.0,508.0,41.0,1083.0,549.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
3,4,Sun,0.0,2.0,1.0,Cleveland Browns,2019,Ravens,110.0,60.0,...,3.0,871.0,181.0,1535.0,1052.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
4,5,Sun,1.0,2.0,0.0,Pittsburgh Steelers,2019,Ravens,135.0,100.0,...,4.0,1208.0,374.0,1930.0,1582.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,8.0,1.0,Pittsburgh Steelers,2010,Ravens,250.0,188.0,...,14.0,2262.0,1119.0,3831.0,3381.0,0,John Harbaugh,Cam Cameron,Greg Mattison,Joe Flacco
156,14,Mon,1.0,8.0,0.0,Houston Texans,2010,Ravens,260.0,201.0,...,15.0,2496.0,1173.0,4100.0,3669.0,0,John Harbaugh,Cam Cameron,Greg Mattison,Joe Flacco
157,15,Sun,1.0,9.0,1.0,New Orleans Saints,2010,Ravens,294.0,229.0,...,17.0,2874.0,1284.0,4353.0,4158.0,0,John Harbaugh,Cam Cameron,Greg Mattison,Joe Flacco
158,16,Sun,1.0,10.0,0.0,Cleveland Browns,2010,Ravens,324.0,253.0,...,18.0,3116.0,1311.0,4709.0,4427.0,0,John Harbaugh,Cam Cameron,Greg Mattison,Joe Flacco


In [395]:
ravens_df.to_csv('Resources/indv_team_csvs/ravens.csv')

### (15) Eagles Data, 2010-2019

In [396]:
# Data Source: https://www.pro-football-reference.com/teams/phi/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

eagles_df2010 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2010.csv')
eagles_df2011 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2011.csv')
eagles_df2012 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2012.csv')
eagles_df2013 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2013.csv')
eagles_df2014 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2014.csv')
eagles_df2015 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2015.csv')
eagles_df2016 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2016.csv')
eagles_df2017 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2017.csv')
eagles_df2018 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2018.csv')
eagles_df2019 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2019.csv')

eagles_df2010 = yearly_data(df=eagles_df2010, year=2010, 
                        HC='Andy Reid', 
                        OC='Marty Mornhinweg', 
                        DC='Sean McDermott', 
                        QB='Michael Vick', 
                        QB2='Kevin Kolb', QB2_range=range(4,7,1))
eagles_df2010.loc[eagles_df2010['Week'] == range(1,18,16), 'QB'] = "Kevin Kolb"

eagles_df2011 = yearly_data(df=eagles_df2011, year=2011, 
                        HC='Andy Reid', 
                        OC='Marty Mornhinweg', 
                        DC='Juan Castillo', 
                        QB='Michael Vick', 
                        QB2='Vince Young', QB2_range=range(11,14,1))

eagles_df2012 = yearly_data(df=eagles_df2012, year=2012, 
                        HC='Andy Reid', 
                        OC='Marty Mornhinweg', 
                        DC='Juan Castillo',
                        QB='Michael Vick', 
                        QB2='Nick Foles', QB2_range=range(11,17,1))
eagles_df2012.loc[eagles_df2012['Week'] == range(8,18,1), 'DC'] = "Todd Bowles"

eagles_df2013 = yearly_data(df=eagles_df2013, year=2013, 
                        HC='Chip Kelly', 
                        OC='Pat Shurmur', 
                        DC='Bill Davis', 
                        QB='Nick Foles', 
                        QB2='Michael Vick', QB2_range=range(1,6,1))
eagles_df2013.loc[eagles_df2013['Week'] == 8, 'QB'] = "Michael Vick"

eagles_df2014 = yearly_data(df=eagles_df2014, year=2014, 
                        HC='Chip Kelly', 
                        OC='Pat Shurmur', 
                        DC='Bill Davis', 
                        QB='Nick Foles', 
                        QB2='Mark Sanchez', QB2_range=range(10,18,1))

eagles_df2015 = yearly_data(df=eagles_df2015, year=2015, 
                        HC='Chip Kelly', HC2='Pat Shurmur', HC2_range=17, 
                        OC='Pat Shurmur', 
                        DC='Bill Davis', 
                        QB='Sam Bradford', 
                        QB2='Mark Sanchez', QB2_range=range(11,13,1))

eagles_df2016 = yearly_data(df=eagles_df2016, year=2016, 
                        HC='Doug Pederson',
                        OC='Frank Reich', 
                        DC="Jim Schwartz", 
                        QB='Carson Wentz')

eagles_df2017 = yearly_data(df=eagles_df2017, year=2017, 
                        HC='Doug Pederson',
                        OC='Frank Reich', 
                        DC="Jim Schwartz", 
                        QB='Carson Wentz', 
                        QB2='Nick Foles', QB2_range=range(15,18,1))

eagles_df2018 = yearly_data(df=eagles_df2018, year=2018, 
                        HC='Doug Pederson',
                        OC='Mike Groh', 
                        DC="Jim Schwartz", 
                        QB='Carson Wentz', 
                        QB2='Nick Foles', QB2_range=range(15,18,1))
eagles_df2018.loc[eagles_df2018['Week'] == range(1,3,1), 'QB'] = "Nick Foles"

eagles_df2019 = yearly_data(df=eagles_df2019, year=2019, 
                        HC='Doug Pederson',
                        OC='Mike Groh', 
                        DC="Jim Schwartz", 
                        QB='Carson Wentz')

# Make master dataframe for team
eagles_df = eagles_df2019.append(eagles_df2018, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2017, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2016, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2015, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2014, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2013, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2012, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2011, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2010, ignore_index=True)

eagles_df['Base_Team'] = 'Eagles'
eagles_df = eagles_df.sort_values(by=['year', 'Week'], ascending=[False, True])

eagles_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Washington Redskins,2019,Eagles,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Doug Pederson,Mike Groh,Jim Schwartz,Carson Wentz
1,2,Sun,0.0,1.0,0.0,Atlanta Falcons,2019,Eagles,32.0,27.0,...,0.0,370.0,28.0,436.0,398.0,0,Doug Pederson,Mike Groh,Jim Schwartz,Carson Wentz
2,3,Sun,0.0,1.0,1.0,Detroit Lions,2019,Eagles,52.0,51.0,...,3.0,680.0,85.0,722.0,765.0,0,Doug Pederson,Mike Groh,Jim Schwartz,Carson Wentz
3,4,Thu,1.0,1.0,0.0,Green Bay Packers,2019,Eagles,76.0,78.0,...,3.0,881.0,171.0,1095.0,1052.0,0,Doug Pederson,Mike Groh,Jim Schwartz,Carson Wentz
4,5,Sun,1.0,2.0,1.0,New York Jets,2019,Eagles,110.0,105.0,...,5.0,1295.0,248.0,1431.0,1543.0,0,Doug Pederson,Mike Groh,Jim Schwartz,Carson Wentz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Thu,1.0,7.0,1.0,Houston Texans,2010,Eagles,310.0,257.0,...,26.0,2354.0,1128.0,4392.0,3482.0,0,Andy Reid,Marty Mornhinweg,Sean McDermott,Michael Vick
156,14,Sun,1.0,8.0,0.0,Dallas Cowboys,2010,Eagles,344.0,281.0,...,28.0,2677.0,1236.0,4808.0,3913.0,0,Andy Reid,Marty Mornhinweg,Sean McDermott,Michael Vick
157,15,Sun,1.0,9.0,0.0,New York Giants,2010,Eagles,374.0,308.0,...,30.0,2916.0,1346.0,5237.0,4262.0,0,Andy Reid,Marty Mornhinweg,Sean McDermott,Michael Vick
158,16,Tue,0.0,10.0,1.0,Minnesota Vikings,2010,Eagles,412.0,339.0,...,32.0,3180.0,1446.0,5655.0,4626.0,0,Andy Reid,Marty Mornhinweg,Sean McDermott,Michael Vick


In [397]:
eagles_df.to_csv('Resources/indv_team_csvs/eagles.csv')

### (16) Cowboys Data, 2010-2019

In [398]:
# Data Source: https://www.pro-football-reference.com/teams/dal/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

cowboys_df2010 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2010.csv')
cowboys_df2011 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2011.csv')
cowboys_df2012 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2012.csv')
cowboys_df2013 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2013.csv')
cowboys_df2014 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2014.csv')
cowboys_df2015 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2015.csv')
cowboys_df2016 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2016.csv')
cowboys_df2017 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2017.csv')
cowboys_df2018 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2018.csv')
cowboys_df2019 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2019.csv')

cowboys_df2010 = yearly_data(df=cowboys_df2010, year=2010, 
                        HC='Jason Garrett', HC2='Wade Phillips', HC2_range=range(1,10,1), 
                        OC='Jason Garrett', 
                        DC='Paul Pasqualoni', 
                        QB='Jon Kitna', 
                        QB2='Tony Romo', QB2_range=range(1,8,1))
cowboys_df2010.loc[cowboys_df2010['Week'] == 17, 'QB'] = "Stephen McGee"

cowboys_df2011 = yearly_data(df=cowboys_df2011, year=2011, 
                          HC='Jason Garrett', 
                          OC='Jason Garrett', 
                          DC='Rob Ryan', 
                          QB='Tony Romo')

cowboys_df2012 = yearly_data(df=cowboys_df2012, year=2012, 
                        HC='Jason Garrett', 
                        OC='Bill Callahan', 
                        DC='Rob Ryan',
                        QB='Tony Romo')

cowboys_df2013 = yearly_data(df=cowboys_df2013, year=2013, 
                        HC='Jason Garrett', 
                        OC='Bill Callahan', 
                        DC='Monte Kiffin', 
                        QB='Tony Romo',
                        QB2='Kyle Orton', QB2_range=17)

cowboys_df2014 = yearly_data(df=cowboys_df2014, year=2014, 
                        HC='Jason Garrett', 
                        OC='Bill Callahan', 
                        DC='Rod Marinelli', 
                        QB='Tony Romo', 
                        QB2='Brandon Weeden', QB2_range=9)

cowboys_df2015 = yearly_data(df=cowboys_df2015, year=2015, 
                        HC='Jason Garrett', 
                        OC='Scott Linehan', 
                        DC='Rod Marinelli', 
                        QB='Matt Cassel', 
                        QB2='Tony Romo', QB2_range=range(1,3,1))
cowboys_df2015.loc[cowboys_df2015['Week'] == range(3,6,1), 'QB'] = "Brandon Weeden"
cowboys_df2015.loc[cowboys_df2015['Week'] == range(16,18,1), 'QB'] = "Kellen Moore"
cowboys_df2015.loc[cowboys_df2015['Week'] == range(11,13,1), 'QB'] = "Tony Romo"

cowboys_df2016 = yearly_data(df=cowboys_df2016, year=2016, 
                        HC='Jason Garrett',
                        OC='Scott Linehan', 
                        DC="Rod Marinelli", 
                        QB='Dak Prescott')

cowboys_df2017 = yearly_data(df=cowboys_df2017, year=2017, 
                        HC='Jason Garrett', 
                        OC='Scott Linehan', 
                        DC="Rod Marinelli", 
                        QB='Dak Prescott')

cowboys_df2018 = yearly_data(df=cowboys_df2018, year=2018, 
                        HC='Jason Garrett', 
                        OC='Scott Linehan', 
                        DC="Rod Marinelli", 
                        QB='Dak Prescott')

cowboys_df2019 = yearly_data(df=cowboys_df2019, year=2019, 
                        HC='Jason Garrett', 
                        OC='Kellen Moore', 
                        DC='Rod Marinelli', 
                        QB='Dak Prescott')

# Make master dataframe for team
cowboys_df = cowboys_df2019.append(cowboys_df2018, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2017, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2016, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2015, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2014, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2013, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2012, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2011, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2010, ignore_index=True)

cowboys_df['Base_Team'] = 'Cowboys'
cowboys_df = cowboys_df.sort_values(by=['year', 'Week'], ascending=[False, True])

cowboys_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,New York Giants,2019,Cowboys,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Jason Garrett,Kellen Moore,Rod Marinelli,Dak Prescott
1,2,Sun,1.0,1.0,0.0,Washington Redskins,2019,Cowboys,35.0,17.0,...,2.0,319.0,151.0,494.0,470.0,0,Jason Garrett,Kellen Moore,Rod Marinelli,Dak Prescott
2,3,Sun,1.0,2.0,1.0,Miami Dolphins,2019,Cowboys,66.0,38.0,...,2.0,527.0,198.0,968.0,725.0,0,Jason Garrett,Kellen Moore,Rod Marinelli,Dak Prescott
3,4,Sun,0.0,3.0,0.0,New Orleans Saints,2019,Cowboys,97.0,44.0,...,3.0,738.0,270.0,1444.0,1008.0,0,Jason Garrett,Kellen Moore,Rod Marinelli,Dak Prescott
4,5,Sun,0.0,3.0,1.0,Green Bay Packers,2019,Cowboys,107.0,56.0,...,4.0,887.0,387.0,1701.0,1274.0,0,Jason Garrett,Kellen Moore,Rod Marinelli,Dak Prescott
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,3.0,0.0,Indianapolis Colts,2010,Cowboys,256.0,301.0,...,17.0,2684.0,1254.0,4023.0,3938.0,0,Jason Garrett,Jason Garrett,Paul Pasqualoni,Jon Kitna
156,14,Sun,0.0,4.0,1.0,Philadelphia Eagles,2010,Cowboys,294.0,336.0,...,21.0,3049.0,1294.0,4391.0,4343.0,0,Jason Garrett,Jason Garrett,Paul Pasqualoni,Jon Kitna
157,15,Sun,1.0,4.0,1.0,Washington Redskins,2010,Cowboys,321.0,366.0,...,23.0,3307.0,1465.0,4740.0,4772.0,0,Jason Garrett,Jason Garrett,Paul Pasqualoni,Jon Kitna
158,16,Sat,0.0,5.0,0.0,Arizona Cardinals,2010,Cowboys,354.0,396.0,...,26.0,3593.0,1520.0,5174.0,5113.0,0,Jason Garrett,Jason Garrett,Paul Pasqualoni,Jon Kitna


In [399]:
cowboys_df.to_csv('Resources/indv_team_csvs/cowboys.csv')

### (17) Chiefs Data, 2010-2019

In [400]:
# Data Source: https://www.pro-football-reference.com/teams/kan/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

chiefs_df2010 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2010.csv')
chiefs_df2011 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2011.csv')
chiefs_df2012 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2012.csv')
chiefs_df2013 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2013.csv')
chiefs_df2014 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2014.csv')
chiefs_df2015 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2015.csv')
chiefs_df2016 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2016.csv')
chiefs_df2017 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2017.csv')
chiefs_df2018 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2018.csv')
chiefs_df2019 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2019.csv')

chiefs_df2010 = yearly_data(df=chiefs_df2010, year=2010, 
                        HC='Todd Haley', 
                        OC='Charlie Weis', 
                        DC='Romeo Crennel', 
                        QB='Matt Cassel', 
                        QB2='Brodie Croyle', QB2_range=14)

chiefs_df2011 = yearly_data(df=chiefs_df2011, year=2011, 
                        HC='Todd Haley', HC2='Romeo Crennel', HC2_range=range(15,18,1), 
                        OC='Bill Muir', 
                        DC='Romeo Crennel', 
                        QB='Matt Cassel', 
                        QB2='Tyler Palko', QB2_range=range(11,15,1))
chiefs_df2011.loc[chiefs_df2011['Week'] == range(15,18,1), 'QB'] = "Kyle Orton"

chiefs_df2012 = yearly_data(df=chiefs_df2012, year=2012, 
                        HC='Romeo Crennel', 
                        OC='Brian Daboll', 
                        DC='Romeo Crennel',
                        QB='Matt Cassel', 
                        QB2='Brady Quinn', QB2_range=range(6,9,1))
chiefs_df2012.loc[chiefs_df2012['Week'] == range(12,18,1), 'QB'] = "Brady Quinn"

chiefs_df2013 = yearly_data(df=chiefs_df2013, year=2013, 
                        HC='Andy Reid', 
                        OC='Doug Pederson', 
                        DC='Bob Sutton', 
                        QB='Alex Smith', 
                        QB2='Chase Daniel', QB2_range=17)

chiefs_df2014 = yearly_data(df=chiefs_df2014, year=2014, 
                        HC='Andy Reid', 
                        OC='Doug Pederson', 
                        DC='Bob Sutton', 
                        QB='Alex Smith', 
                        QB2='Chase Daniel', QB2_range=17)

chiefs_df2015 = yearly_data(df=chiefs_df2015, year=2015, 
                        HC='Andy Reid', 
                        OC='Doug Pederson', 
                        DC='Bob Sutton', 
                        QB='Alex Smith')

chiefs_df2016 = yearly_data(df=chiefs_df2016, year=2016, 
                        HC='Andy Reid', 
                        OC='Brad Childress', 
                        DC='Bob Sutton', 
                        QB='Alex Smith', 
                        QB2='Nick Foles', QB2_range=9)

chiefs_df2017 = yearly_data(df=chiefs_df2017, year=2017, 
                        HC='Andy Reid', 
                        OC='Matt Nagy', 
                        DC='Bob Sutton', 
                        QB='Alex Smith', 
                        QB2='Patrick Mahomes', QB2_range=17)

chiefs_df2018 = yearly_data(df=chiefs_df2018, year=2018, 
                        HC='Andy Reid', 
                        OC='Eric Bieniemy', 
                        DC='Bob Sutton', 
                        QB='Patrick Mahomes')

chiefs_df2019 = yearly_data(df=chiefs_df2019, year=2019, 
                        HC='Andy Reid', 
                        OC='Eric Bieniemy', 
                        DC='Steve Spagnuolo', 
                        QB='Patrick Mahomes', 
                        QB2='Matt Moore', QB2_range=range(8,10,1))

# Make master dataframe for team
chiefs_df = chiefs_df2019.append(chiefs_df2018, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2017, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2016, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2015, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2014, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2013, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2012, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2011, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2010, ignore_index=True)

chiefs_df['Base_Team'] = 'Chiefs'
chiefs_df = chiefs_df.sort_values(by=['year', 'Week'], ascending=[False, True])

chiefs_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Jacksonville Jaguars,2019,Chiefs,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
1,2,Sun,1.0,1.0,0.0,Oakland Raiders,2019,Chiefs,40.0,26.0,...,2.0,347.0,81.0,491.0,428.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
2,3,Sun,1.0,2.0,1.0,Baltimore Ravens,2019,Chiefs,68.0,36.0,...,4.0,525.0,210.0,958.0,735.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
3,4,Sun,1.0,3.0,0.0,Detroit Lions,2019,Chiefs,101.0,64.0,...,4.0,774.0,413.0,1461.0,1187.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
4,5,Sun,0.0,4.0,1.0,Indianapolis Colts,2019,Chiefs,135.0,94.0,...,6.0,1035.0,599.0,1899.0,1634.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,7.0,1.0,Denver Broncos,2010,Chiefs,285.0,231.0,...,15.0,2711.0,1061.0,4127.0,3772.0,0,Todd Haley,Charlie Weis,Romeo Crennel,Matt Cassel
156,14,Sun,0.0,8.0,0.0,San Diego Chargers,2010,Chiefs,295.0,237.0,...,16.0,2797.0,1222.0,4486.0,4019.0,0,Brodie Croyle,Charlie Weis,Romeo Crennel,Matt Cassel
157,15,Sun,1.0,8.0,0.0,St. Louis Rams,2010,Chiefs,295.0,268.0,...,18.0,3016.0,1429.0,4553.0,4445.0,0,Todd Haley,Charlie Weis,Romeo Crennel,Matt Cassel
158,16,Sun,1.0,9.0,1.0,Tennessee Titans,2010,Chiefs,322.0,281.0,...,20.0,3171.0,1498.0,4936.0,4669.0,0,Todd Haley,Charlie Weis,Romeo Crennel,Matt Cassel


In [401]:
chiefs_df.to_csv('Resources/indv_team_csvs/chiefs.csv')

### Lions Data, 2010-2019

In [402]:
# Data Source: https://www.pro-football-reference.com/teams/det/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

lions_df2010 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2010.csv')
lions_df2011 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2011.csv')
lions_df2012 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2012.csv')
lions_df2013 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2013.csv')
lions_df2014 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2014.csv')
lions_df2015 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2015.csv')
lions_df2016 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2016.csv')
lions_df2017 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2017.csv')
lions_df2018 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2018.csv')
lions_df2019 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2019.csv')

lions_df2010 = yearly_data(df=lions_df2010, year=2010, 
                        HC='Jim Schwartz', 
                        OC='Scott Linehan', 
                        DC='Gunther Cunningham', 
                        QB='Shaun Hill', 
                        QB2='Matthew Stafford', QB2_range=range(8,10,1))
lions_df2010.loc[lions_df2010['Week'] == range(13,16,1), 'QB'] = "Drew Stanton"
lions_df2010.loc[lions_df2010['Week'] == 1, 'QB'] = "Matthew Stafford"

lions_df2011 = yearly_data(df=lions_df2011, year=2011, 
                        HC='Jim Schwartz', 
                        OC='Scott Linehan', 
                        DC='Gunther Cunningham', 
                        QB='Matthew Stafford')

lions_df2012 = yearly_data(df=lions_df2012, year=2012, 
                        HC='Jim Schwartz', 
                        OC='Scott Linehan', 
                        DC='Gunther Cunningham',
                        QB='Matthew Stafford')

lions_df2013 = yearly_data(df=lions_df2013, year=2013, 
                        HC='Jim Schwartz', 
                        OC='Scott Linehan', 
                        DC='Gunther Cunningham', 
                        QB='Matthew Stafford')

lions_df2014 = yearly_data(df=lions_df2014, year=2014, 
                        HC='Jim Caldwell', 
                        OC='Joe Lombardi', 
                        DC='Jim Bob Cooter', 
                        QB='Matthew Stafford')

lions_df2015 = yearly_data(df=lions_df2015, year=2015, 
                        HC='Jim Caldwell', 
                        OC='Joe Lombardi', 
                        DC='Teryl Austin', 
                        QB='Matthew Stafford')
lions_df2015.loc[lions_df2015['Week'] == range(7,18,1), 'OC'] = 'Jim Bob Cooter'

lions_df2016 = yearly_data(df=lions_df2016, year=2016, 
                        HC='Jim Caldwell',
                        OC='Jim Bob Cooter', 
                        DC="Teryl Austin", 
                        QB='Matthew Stafford')

lions_df2017 = yearly_data(df=lions_df2017, year=2017, 
                        HC='Jim Caldwell', 
                        OC='Jim Bob Cooter', 
                        DC='Teryl Austin', 
                        QB='Matthew Stafford')

lions_df2018 = yearly_data(df=lions_df2018, year=2018, 
                        HC='Matt Patricia', 
                        OC='Jim Bob Cooter', 
                        DC='Paul Pasqualoni', 
                        QB='Matthew Stafford')

lions_df2019 = yearly_data(df=lions_df2019, year=2019, 
                        HC='Matt Patricia', 
                        OC='Darrell Bevell', 
                        DC='Paul Pasqualoni', 
                        QB='Matthew Stafford',
                        QB2='David Blough', QB2_range=range(13,18,1))
lions_df2019.loc[lions_df2019['Week'] == range(10,13,1), 'QB'] = "Jeff Driskel"

# Make master dataframe for team
lions_df = lions_df2019.append(lions_df2018, ignore_index=True)
lions_df = lions_df.append(lions_df2017, ignore_index=True)
lions_df = lions_df.append(lions_df2016, ignore_index=True)
lions_df = lions_df.append(lions_df2015, ignore_index=True)
lions_df = lions_df.append(lions_df2014, ignore_index=True)
lions_df = lions_df.append(lions_df2013, ignore_index=True)
lions_df = lions_df.append(lions_df2012, ignore_index=True)
lions_df = lions_df.append(lions_df2011, ignore_index=True)
lions_df = lions_df.append(lions_df2010, ignore_index=True)

lions_df['Base_Team'] = 'Lions'
lions_df = lions_df.sort_values(by=['year', 'Week'], ascending=[False, True])

lions_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Arizona Cardinals,2019,Lions,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Matt Patricia,Darrell Bevell,Paul Pasqualoni,Matthew Stafford
1,2,Sun,1.0,0.0,1.0,Los Angeles Chargers,2019,Lions,27.0,27.0,...,1.0,275.0,112.0,477.0,387.0,0,Matt Patricia,Darrell Bevell,Paul Pasqualoni,Matthew Stafford
2,3,Sun,1.0,1.0,0.0,Philadelphia Eagles,2019,Lions,40.0,37.0,...,3.0,562.0,249.0,816.0,811.0,0,Matt Patricia,Darrell Bevell,Paul Pasqualoni,Matthew Stafford
3,4,Sun,0.0,2.0,1.0,Kansas City Chiefs,2019,Lions,67.0,61.0,...,5.0,808.0,376.0,1103.0,1184.0,0,Matt Patricia,Darrell Bevell,Paul Pasqualoni,Matthew Stafford
4,6,Mon,0.0,2.0,0.0,Green Bay Packers,2019,Lions,97.0,95.0,...,8.0,1123.0,499.0,1550.0,1622.0,1,Matt Patricia,Darrell Bevell,Paul Pasqualoni,Matthew Stafford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Thu,1.0,6.0,1.0,Green Bay Packers,2010,Lions,286.0,277.0,...,14.0,3045.0,968.0,4533.0,4013.0,0,Jim Schwartz,Scott Linehan,Gunther Cunningham,Shaun Hill
156,14,Sun,0.0,7.0,0.0,Philadelphia Eagles,2010,Lions,326.0,287.0,...,17.0,3147.0,992.0,5094.0,4139.0,0,Jim Schwartz,Scott Linehan,Gunther Cunningham,Shaun Hill
157,15,Mon,0.0,7.0,1.0,Baltimore Ravens,2010,Lions,346.0,321.0,...,18.0,3326.0,1291.0,5325.0,4617.0,0,Jim Schwartz,Scott Linehan,Gunther Cunningham,Shaun Hill
158,16,Sun,0.0,7.0,1.0,New York Giants,2010,Lions,362.0,339.0,...,18.0,3541.0,1381.0,5674.0,4922.0,0,Jim Schwartz,Scott Linehan,Gunther Cunningham,Shaun Hill


In [403]:
lions_df.to_csv('Resources/indv_team_csvs/lions.csv')

### Vikings Data, 2010-2019

In [404]:
# Data Source: https://www.pro-football-reference.com/teams/min/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

vikings_df2010 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2010.csv')
vikings_df2011 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2011.csv')
vikings_df2012 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2012.csv')
vikings_df2013 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2013.csv')
vikings_df2014 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2014.csv')
vikings_df2015 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2015.csv')
vikings_df2016 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2016.csv')
vikings_df2017 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2017.csv')
vikings_df2018 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2018.csv')
vikings_df2019 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2019.csv')

vikings_df2010 = yearly_data(df=vikings_df2010, year=2010, 
                        HC='Brad Childress', HC2='Leslie Frazier', HC2_range=range(13,18,1),
                        OC='Darrell Bevell', 
                        DC='Leslie Frazier', 
                        QB='Brett Favre',
                        QB2='Joe Webb', QB2_range=range(16,18,1))
vikings_df2010.loc[vikings_df2010['Week'] == range(13,18,1), 'DC'] = "Fred Pagac"
vikings_df2010.loc[vikings_df2010['Week'] == 14, 'QB'] = "Tarvaris Jackson"

vikings_df2011 = yearly_data(df=vikings_df2011, year=2011, 
                        HC='Leslie Frazier', 
                        OC='Bill Musgrave', 
                        DC='Fred Pagac', 
                        QB='Christian Ponder',
                        QB2='Donovan McNabb', QB2_range=range(1,7,1))

vikings_df2012 = yearly_data(df=vikings_df2012, year=2012, 
                        HC='Leslie Frazier', 
                        OC='Bill Musgrave', 
                        DC='Alan Williams',
                        QB='Christian Ponder')

vikings_df2013 = yearly_data(df=vikings_df2013, year=2013, 
                        HC='Leslie Frazier', 
                        OC='Bill Musgrave', 
                        DC='Alan Williams', 
                        QB='Christian Ponder',
                        QB2='Matt Cassel', QB2_range=range(14,18,1))
vikings_df2013.loc[vikings_df2013['Week'] == range(4,7,1), 'QB'] = "Matt Cassel"
vikings_df2013.loc[vikings_df2013['Week'] == 7, 'QB'] = "Josh Freeman"

vikings_df2014 = yearly_data(df=vikings_df2014, year=2014, 
                        HC='Mike Zimmer', 
                        OC='Norv Turner', 
                        DC='George Edwards', 
                        QB='Teddy Bridgewater',
                        QB2='Matt Cassel', QB2_range=range(1,4,1))
vikings_df2014.loc[vikings_df2014['Week'] == 5, 'QB'] = "Christian Ponder"

vikings_df2015 = yearly_data(df=vikings_df2015, year=2015, 
                        HC='Mike Zimmer', 
                        OC='Norv Turner', 
                        DC='George Edwards', 
                        QB='Teddy Bridgewater')

vikings_df2016 = yearly_data(df=vikings_df2016, year=2016, 
                        HC='Mike Zimmer', HC2='Mike Priefer', HC2_range=13, 
                        OC='Norv Turner', 
                        DC='George Edwards', 
                        QB='Sam Bradford',
                        QB2='Shaun Hill', QB2_range=1)
vikings_df2016.loc[vikings_df2016['Week'] == range(9,18,1), 'OC'] = "Pat Shurmur"

vikings_df2017 = yearly_data(df=vikings_df2017, year=2017, 
                        HC='Mike Zimmer', 
                        OC='Pat Shurmur', 
                        DC='George Edwards', 
                        QB='Case Keenum',
                        QB2='Sam Bradford', QB2_range=range(1,3,1))

vikings_df2018 = yearly_data(df=vikings_df2018, year=2018, 
                        HC='Mike Zimmer', 
                        OC='John DeFilippo', 
                        DC='George Edwards', 
                        QB='Kirk Cousins')

vikings_df2019 = yearly_data(df=vikings_df2019, year=2019, 
                        HC='Mike Zimmer', 
                        OC='Kevin Stefanski', 
                        DC='George Edwards', 
                        QB='Kirk Cousins',
                        QB2='Sean Mannion', QB2_range=17)

# Make master dataframe for team
vikings_df = vikings_df2019.append(vikings_df2018, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2017, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2016, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2015, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2014, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2013, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2012, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2011, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2010, ignore_index=True)

vikings_df['Base_Team'] = 'Vikings'
vikings_df = vikings_df.sort_values(by=['year', 'Week'], ascending=[False, True])

vikings_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Atlanta Falcons,2019,Vikings,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Mike Zimmer,Kevin Stefanski,George Edwards,Kirk Cousins
1,2,Sun,0.0,1.0,0.0,Green Bay Packers,2019,Vikings,28.0,12.0,...,3.0,272.0,73.0,269.0,345.0,0,Mike Zimmer,Kevin Stefanski,George Edwards,Kirk Cousins
2,3,Sun,1.0,1.0,1.0,Oakland Raiders,2019,Vikings,44.0,33.0,...,5.0,463.0,217.0,690.0,680.0,0,Mike Zimmer,Kevin Stefanski,George Edwards,Kirk Cousins
3,4,Sun,0.0,2.0,0.0,Chicago Bears,2019,Vikings,78.0,47.0,...,6.0,677.0,305.0,1075.0,982.0,0,Mike Zimmer,Kevin Stefanski,George Edwards,Kirk Cousins
4,5,Sun,1.0,2.0,0.0,New York Giants,2019,Vikings,84.0,63.0,...,6.0,874.0,377.0,1297.0,1251.0,0,Mike Zimmer,Kevin Stefanski,George Edwards,Kirk Cousins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,4.0,1.0,Buffalo Bills,2010,Vikings,189.0,239.0,...,13.0,2367.0,1020.0,3667.0,3387.0,0,Brad Childress,Darrell Bevell,Leslie Frazier,Brett Favre
156,14,Mon,0.0,5.0,1.0,New York Giants,2010,Vikings,227.0,253.0,...,18.0,2522.0,1104.0,4054.0,3626.0,0,Brad Childress,Darrell Bevell,Leslie Frazier,Tarvaris Jackson
157,15,Mon,0.0,5.0,1.0,Chicago Bears,2010,Vikings,230.0,274.0,...,20.0,2704.0,1317.0,4218.0,4021.0,0,Brad Childress,Darrell Bevell,Leslie Frazier,Brett Favre
158,16,Tue,1.0,5.0,0.0,Philadelphia Eagles,2010,Vikings,244.0,314.0,...,21.0,2893.0,1421.0,4491.0,4314.0,0,Brad Childress,Darrell Bevell,Leslie Frazier,Brett Favre


In [405]:
vikings_df.to_csv('Resources/indv_team_csvs/vikings.csv')

### (20) Bengals Data, 2010-2019

In [406]:
# Data Source: https://www.pro-football-reference.com/teams/cin/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

bengals_df2010 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2010.csv')
bengals_df2011 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2011.csv')
bengals_df2012 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2012.csv')
bengals_df2013 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2013.csv')
bengals_df2014 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2014.csv')
bengals_df2015 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2015.csv')
bengals_df2016 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2016.csv')
bengals_df2017 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2017.csv')
bengals_df2018 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2018.csv')
bengals_df2019 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2019.csv')

# Variables for yearly_data formula:
# yearly_data(df, year, HC2_range=0, QB2_range=0, HC='', HC2='none', OC='', DC='', QB='', QB2='none')

bengals_df2010 = yearly_data(df=bengals_df2010, year=2010, 
                        HC='Marvin Lewis', 
                        OC='Bob Bratkowski', 
                        DC='Mike Zimmer', 
                        QB='Carson Palmer')

bengals_df2011 = yearly_data(df=bengals_df2011, year=2011, 
                        HC='Marvin Lewis', 
                        OC='Jay Gruden', 
                        DC='Mike Zimmer', 
                        QB='Andy Dalton')

bengals_df2012 = yearly_data(df=bengals_df2012, year=2012, 
                        HC='Marvin Lewis', 
                        OC='Jay Gruden', 
                        DC='Mike Zimmer',
                        QB='Andy Dalton')

bengals_df2013 = yearly_data(df=bengals_df2013, year=2013, 
                        HC='Marvin Lewis', 
                        OC='Jay Gruden', 
                        DC='Mike Zimmer', 
                        QB='Andy Dalton')

bengals_df2014 = yearly_data(df=bengals_df2014, year=2014, 
                        HC='Marvin Lewis', 
                        OC='Hue Jackson', 
                        DC='Paul Guenther', 
                        QB='Andy Dalton')

bengals_df2015 = yearly_data(df=bengals_df2015, year=2015, 
                        HC='Marvin Lewis', 
                        OC='Hue Jackson', 
                        DC='Paul Guenther', 
                        QB='Andy Dalton',
                        QB2='AJ McCarron', QB2_range=range(15,18,1))

bengals_df2016 = yearly_data(df=bengals_df2016, year=2016, 
                        HC='Marvin Lewis',
                        OC='Ken Zampese', 
                        DC="Paul Guenther", 
                        QB='Andy Dalton')

bengals_df2017 = yearly_data(df=bengals_df2017, year=2017, 
                        HC='Marvin Lewis', 
                        OC='Ken Zampese', 
                        DC='Paul Guenther', 
                        QB='Andy Dalton')
bengals_df2017.loc[bengals_df2017['Week'] == range(3,18,1), 'OC'] = "Bill Lazor"

bengals_df2018 = yearly_data(df=bengals_df2018, year=2018, 
                        HC='Marvin Lewis', 
                        OC='Bill Lazor', 
                        DC='Teryl Austin', 
                        QB='Andy Dalton',
                        QB2='Jeff Driskel', QB2_range=range(13,18,1))

bengals_df2019 = yearly_data(df=bengals_df2019, year=2019, 
                        HC='Zac Taylor', 
                        OC='Brian Callahan', 
                        DC='Lou Anarumo', 
                        QB='Andy Dalton',
                        QB2='Ryan Finley', QB2_range=range(10,13,1))

# Make master dataframe for team
bengals_df = bengals_df2019.append(bengals_df2018, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2017, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2016, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2015, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2014, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2013, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2012, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2011, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2010, ignore_index=True)

bengals_df['Base_Team'] = 'Bengals'
bengals_df = bengals_df.sort_values(by=['year', 'Week'], ascending=[False, True])

bengals_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Seattle Seahawks,2019,Bengals,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Andy Dalton
1,2,Sun,0.0,0.0,1.0,San Francisco 49ers,2019,Bengals,20.0,21.0,...,1.0,160.0,72.0,429.0,232.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Andy Dalton
2,3,Sun,0.0,0.0,0.0,Buffalo Bills,2019,Bengals,37.0,62.0,...,2.0,472.0,331.0,745.0,803.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Andy Dalton
3,4,Mon,0.0,0.0,0.0,Pittsburgh Steelers,2019,Bengals,54.0,83.0,...,4.0,713.0,506.0,1051.0,1219.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Andy Dalton
4,5,Sun,0.0,0.0,1.0,Arizona Cardinals,2019,Bengals,57.0,110.0,...,5.0,973.0,572.0,1226.0,1545.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Andy Dalton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,2.0,1.0,New Orleans Saints,2010,Bengals,225.0,288.0,...,21.0,2379.0,1353.0,3625.0,3732.0,0,Marvin Lewis,Bob Bratkowski,Mike Zimmer,Carson Palmer
156,14,Sun,0.0,2.0,0.0,Pittsburgh Steelers,2010,Bengals,255.0,322.0,...,22.0,2669.0,1499.0,3936.0,4168.0,0,Marvin Lewis,Bob Bratkowski,Mike Zimmer,Carson Palmer
157,15,Sun,1.0,2.0,1.0,Cleveland Browns,2010,Bengals,262.0,345.0,...,22.0,2900.0,1622.0,4126.0,4522.0,0,Marvin Lewis,Bob Bratkowski,Mike Zimmer,Carson Palmer
158,16,Sun,1.0,3.0,1.0,San Diego Chargers,2010,Bengals,281.0,362.0,...,22.0,3119.0,1681.0,4523.0,4800.0,0,Marvin Lewis,Bob Bratkowski,Mike Zimmer,Carson Palmer


In [407]:
bengals_df.to_csv('Resources/indv_team_csvs/bengals.csv')

### Raiders Data, 2010-2019

In [408]:
# Data Source: https://www.pro-football-reference.com/teams/rai/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

raiders_df2010 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2010.csv')
raiders_df2011 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2011.csv')
raiders_df2012 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2012.csv')
raiders_df2013 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2013.csv')
raiders_df2014 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2014.csv')
raiders_df2015 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2015.csv')
raiders_df2016 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2016.csv')
raiders_df2017 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2017.csv')
raiders_df2018 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2018.csv')
raiders_df2019 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2019.csv')

raiders_df2010 = yearly_data(df=raiders_df2010, year=2010, 
                        HC='Tom Cable', 
                        OC='Hue Jackson', 
                        DC='John Marshall', 
                        QB='Jason Campbell', 
                        QB2='Bruce Gradkowski', QB2_range=range(3,6,1))
raiders_df2010.loc[raiders_df2010['Week'] == 12, 'QB'] = "Bruce Gradkowski"

raiders_df2011 = yearly_data(df=raiders_df2011, year=2011, 
                        HC='Hue Jackson', 
                        OC='Al Saunders', 
                        DC='Chuck Bresnahan', 
                        QB='Carson Palmer', 
                        QB2='Jason Campbell', QB2_range=range(1,7,1))
raiders_df2011.loc[raiders_df2011['Week'] == 7, 'QB'] = "Kyle Boller"

raiders_df2012 = yearly_data(df=raiders_df2012, year=2012, 
                        HC='Dennis Allen', 
                        OC='Greg Knapp', 
                        DC='Jason Tarver',
                        QB='Carson Palmer', 
                        QB2='Terrelle Pryor', QB2_range=17)

raiders_df2013 = yearly_data(df=raiders_df2013, year=2013, 
                        HC='Dennis Allen', 
                        OC='Greg Olson', 
                        DC='Jason Tarver', 
                        QB='Terrelle Pryor', 
                        QB2='Matt McGloin', QB2_range=range(11,17,1))
raiders_df2013.loc[raiders_df2013['Week'] == 4, 'QB'] = "Matt Flynn"

raiders_df2014 = yearly_data(df=raiders_df2014, year=2014,  
                        HC='Dennis Allen', HC2='Tony Sparano', HC2_range=range(5,18,1),
                        OC='Greg Olson', 
                        DC='Jason Tarver', 
                        QB='Derek Carr')

raiders_df2015 = yearly_data(df=raiders_df2015, year=2015, 
                        HC='Jack Del Rio', 
                        OC='Bill Musgrave', 
                        DC='Ken Norton', 
                        QB='Derek Carr')

raiders_df2016 = yearly_data(df=raiders_df2016, year=2016, 
                        HC='Jack Del Rio', 
                        OC='Bill Musgrave', 
                        DC='Ken Norton', 
                        QB='Derek Carr', 
                        QB2='Matt McGloin', QB2_range=17)

raiders_df2017 = yearly_data(df=raiders_df2017, year=2017, 
                        HC='Jack Del Rio', 
                        OC='Todd Downing', 
                        DC='Ken Norton', 
                        QB='Derek Carr', 
                        QB2='EJ Manuel', QB2_range=5)
raiders_df2017.loc[raiders_df2017['Week'] == range(12,18,1), 'DC'] = "John Pagano"

raiders_df2018 = yearly_data(df=raiders_df2018, year=2018, 
                        HC='Jon Gruden', 
                        OC='Greg Olson', 
                        DC='Paul Guenther', 
                        QB='Derek Carr')

raiders_df2019 = yearly_data(df=raiders_df2019, year=2019, 
                        HC='Jon Gruden', 
                        OC='Greg Olson', 
                        DC='Paul Guenther', 
                        QB='Derek Carr')

# Make master dataframe for team
raiders_df = raiders_df2019.append(raiders_df2018, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2017, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2016, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2015, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2014, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2013, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2012, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2011, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2010, ignore_index=True)

raiders_df['Base_Team'] = 'Raiders'
raiders_df = raiders_df.sort_values(by=['year', 'Week'], ascending=[False, True])

raiders_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Mon,1.0,0.0,1.0,Denver Broncos,2019,Raiders,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
1,2,Sun,0.0,1.0,1.0,Kansas City Chiefs,2019,Raiders,24.0,16.0,...,0.0,249.0,95.0,357.0,344.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
2,3,Sun,0.0,1.0,0.0,Minnesota Vikings,2019,Raiders,34.0,44.0,...,1.0,685.0,126.0,664.0,811.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
3,4,Sun,1.0,1.0,0.0,Indianapolis Colts,2019,Raiders,48.0,78.0,...,1.0,859.0,337.0,966.0,1196.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
4,5,Sun,1.0,2.0,1.0,Chicago Bears,2019,Raiders,79.0,102.0,...,3.0,1124.0,418.0,1343.0,1542.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,5.0,0.0,San Diego Chargers,2010,Raiders,255.0,256.0,...,16.0,2217.0,1471.0,3694.0,3688.0,0,Tom Cable,Hue Jackson,John Marshall,Jason Campbell
156,14,Sun,0.0,6.0,0.0,Jacksonville Jaguars,2010,Raiders,283.0,269.0,...,18.0,2482.0,1492.0,4062.0,3974.0,0,Tom Cable,Hue Jackson,John Marshall,Jason Campbell
157,15,Sun,1.0,6.0,1.0,Denver Broncos,2010,Raiders,314.0,307.0,...,19.0,2633.0,1726.0,4538.0,4359.0,0,Tom Cable,Hue Jackson,John Marshall,Jason Campbell
158,16,Sun,0.0,7.0,1.0,Indianapolis Colts,2010,Raiders,353.0,330.0,...,20.0,2762.0,1832.0,5040.0,4594.0,0,Tom Cable,Hue Jackson,John Marshall,Jason Campbell


In [409]:
raiders_df.to_csv('Resources/indv_team_csvs/raiders.csv')

### Steelers Data, 2010-2019

In [410]:
# Data Source: https://www.pro-football-reference.com/teams/pit/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

steelers_df2010 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2010.csv')
steelers_df2011 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2011.csv')
steelers_df2012 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2012.csv')
steelers_df2013 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2013.csv')
steelers_df2014 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2014.csv')
steelers_df2015 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2015.csv')
steelers_df2016 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2016.csv')
steelers_df2017 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2017.csv')
steelers_df2018 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2018.csv')
steelers_df2019 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2019.csv')

steelers_df2010 = yearly_data(df=steelers_df2010, year=2010, 
                        HC='Mike Tomlin', 
                        OC='Bruce Arians', 
                        DC='Dick LeBeau', 
                        QB='Ben Roethlisberger',
                        QB2='Charlie Batch', QB2_range=range(3,5,1))
steelers_df2010.loc[steelers_df2010['Week'] == range(1,3,1), 'QB'] = "Dennis Dixon"

steelers_df2011 = yearly_data(df=steelers_df2011, year=2011, 
                        HC='Mike Tomlin', 
                        OC='Bruce Arians', 
                        DC='Dick LeBeau', 
                        QB='Ben Roethlisberger',
                        QB2='Charlie Batch', QB2_range=16)

steelers_df2012 = yearly_data(df=steelers_df2012, year=2012, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Dick LeBeau',
                        QB='Ben Roethlisberger',
                        QB2='Charlie Batch', QB2_range=range(12,14,1))
steelers_df2012.loc[steelers_df2012['Week'] == 11, 'QB'] = "Byron Leftwich"

steelers_df2013 = yearly_data(df=steelers_df2013, year=2013, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Dick LeBeau', 
                        QB='Ben Roethlisberger')

steelers_df2014 = yearly_data(df=steelers_df2014, year=2014, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Dick LeBeau', 
                        QB='Ben Roethlisberger')

steelers_df2015 = yearly_data(df=steelers_df2015, year=2015, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger',
                        QB2='Michael Vick', QB2_range=range(4,7,1))
steelers_df2015.loc[steelers_df2015['Week'] == range(7,11,3), 'QB'] = "Landry Jones"

steelers_df2016 = yearly_data(df=steelers_df2016, year=2016, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger',
                        QB2='Landry Jones', QB2_range=range(7,18,10))

steelers_df2017 = yearly_data(df=steelers_df2017, year=2017, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger',
                        QB2='Landry Jones', QB2_range=17)

steelers_df2018 = yearly_data(df=steelers_df2018, year=2018, 
                        HC='Mike Tomlin', 
                        OC='Randy Fichtner', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger')

steelers_df2019 = yearly_data(df=steelers_df2019, year=2019, 
                        HC='Mike Tomlin', 
                        OC='Randy Fichtner', 
                        DC='Keith Butler', 
                        QB='Mason Rudolph',
                        QB2='Devlin Hodges', QB2_range=6)
steelers_df2019.loc[steelers_df2019['Week'] == range(1,3,1), 'QB'] = "Ben Roethlisberger"
steelers_df2019.loc[steelers_df2019['Week'] == range(13,18,1), 'QB'] = "Devlin Hodges"

# Make master dataframe for team
steelers_df = steelers_df2019.append(steelers_df2018, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2017, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2016, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2015, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2014, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2013, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2012, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2011, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2010, ignore_index=True)

steelers_df['Base_Team'] = 'Steelers'
steelers_df = steelers_df.sort_values(by=['year', 'Week'], ascending=[False, True])

steelers_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,New England Patriots,2019,Steelers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Mason Rudolph
1,2,Sun,0.0,0.0,1.0,Seattle Seahawks,2019,Steelers,3.0,33.0,...,0.0,366.0,99.0,308.0,465.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Mason Rudolph
2,3,Sun,0.0,0.0,0.0,San Francisco 49ers,2019,Steelers,29.0,61.0,...,2.0,640.0,250.0,569.0,890.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Mason Rudolph
3,4,Mon,1.0,0.0,1.0,Cincinnati Bengals,2019,Steelers,49.0,85.0,...,7.0,908.0,418.0,808.0,1326.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Mason Rudolph
4,5,Sun,0.0,1.0,1.0,Baltimore Ravens,2019,Steelers,76.0,88.0,...,9.0,1010.0,491.0,1134.0,1501.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Mason Rudolph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,8.0,0.0,Baltimore Ravens,2010,Steelers,254.0,181.0,...,26.0,2646.0,704.0,3679.0,3350.0,0,Mike Tomlin,Bruce Arians,Dick LeBeau,Ben Roethlisberger
156,14,Sun,1.0,9.0,1.0,Cincinnati Bengals,2010,Steelers,267.0,191.0,...,27.0,2872.0,747.0,3967.0,3619.0,0,Mike Tomlin,Bruce Arians,Dick LeBeau,Ben Roethlisberger
157,15,Sun,0.0,10.0,1.0,New York Jets,2010,Steelers,290.0,198.0,...,30.0,3028.0,781.0,4321.0,3809.0,0,Mike Tomlin,Bruce Arians,Dick LeBeau,Ben Roethlisberger
158,16,Thu,1.0,10.0,1.0,Carolina Panthers,2010,Steelers,307.0,220.0,...,30.0,3198.0,887.0,4699.0,4085.0,0,Mike Tomlin,Bruce Arians,Dick LeBeau,Ben Roethlisberger


In [411]:
steelers_df.to_csv('Resources/indv_team_csvs/steelers.csv')

### Browns Data, 2010-2019

In [412]:
# Data Source: https://www.pro-football-reference.com/teams/cle/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

browns_df2010 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2010.csv')
browns_df2011 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2011.csv')
browns_df2012 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2012.csv')
browns_df2013 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2013.csv')
browns_df2014 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2014.csv')
browns_df2015 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2015.csv')
browns_df2016 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2016.csv')
browns_df2017 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2017.csv')
browns_df2018 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2018.csv')
browns_df2019 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2019.csv')

browns_df2010 = yearly_data(df=browns_df2010, year=2010, 
                        HC='Eric Mangini', 
                        OC='Brian Daboll', 
                        DC='Rob Ryan', 
                        QB='Jake Delhomme', 
                        QB2='Colt McCoy', QB2_range=range(9,12,1))
browns_df2010.loc[browns_df2010['Week'] == range(2,6,1), 'QB'] = "Seneca Wallace"
browns_df2010.loc[browns_df2010['Week'] == range(15,18,1), 'QB'] = "Colt McCoy"

browns_df2011 = yearly_data(df=browns_df2011, year=2011, 
                        HC='Pat Shurmur', 
                        OC='Pat Shurmur', 
                        DC='Dick Jauron', 
                        QB='Colt McCoy', 
                        QB2='Seneca Wallace', QB2_range=range(15,18,1))

browns_df2012 = yearly_data(df=browns_df2012, year=2012, 
                        HC='Pat Shurmur', 
                        OC='Brad Childress', 
                        DC='Dick Jauron',
                        QB='Brandon Weeden', 
                        QB2='Thad Lewis', QB2_range=17)

browns_df2013 = yearly_data(df=browns_df2013, year=2013, 
                        HC='Rob Chudzinski', 
                        OC='Norv Turner', 
                        DC='Ray Horton', 
                        QB='Jason Campbell', 
                        QB2='Brandon Weeden', QB2_range=range(1,8,1))
browns_df2013.loc[browns_df2013['Week'] == range(3,6,1), 'QB'] = "Brian Hoyer"
browns_df2013.loc[browns_df2013['Week'] == 13, 'QB'] = "Brandon Weeden"

browns_df2014 = yearly_data(df=browns_df2014, year=2014, 
                        HC='Mike Pettine', 
                        OC='Kyle Shanahan', 
                        DC="Jim O'Neil", 
                        QB='Brian Hoyer', 
                        QB2='Johnny Manziel', QB2_range=range(15,17,1))
browns_df2014.loc[browns_df2014['Week'] == 17, 'QB'] = "Connor Shaw"

browns_df2015 = yearly_data(df=browns_df2015, year=2015, 
                        HC='Mike Pettine', 
                        OC='John DeFilippo', 
                        DC="Jim O'Neil", 
                        QB='Josh McCown', 
                        QB2='Johnny Manziel', QB2_range=2)
browns_df2015.loc[browns_df2015['Week'] == 13, 'QB'] = "Austin Davis"
browns_df2015.loc[browns_df2015['Week'] == 17, 'QB'] = "Austin Davis"
browns_df2015.loc[browns_df2015['Week'] == range(9,11,1), 'QB'] = "Johnny Manziel"
browns_df2015.loc[browns_df2015['Week'] == range(15,17,1), 'QB'] = "Johnny Manziel"

browns_df2016 = yearly_data(df=browns_df2016, year=2016, 
                        HC='Hue Jackson',
                        OC='Hue Jackson', 
                        DC="Ray Horton", 
                        QB='Cody Kessler', 
                        QB2='Robert Griffin III', QB2_range=range(14,18,1))
browns_df2016.loc[browns_df2016['Week'] == 2, 'QB'] = "Josh McCown"
browns_df2016.loc[browns_df2016['Week'] == 8, 'QB'] = "Josh McCown"
browns_df2016.loc[browns_df2016['Week'] == 12, 'QB'] = "Josh McCown"
browns_df2016.loc[browns_df2016['Week'] == 1, 'QB'] = "Robert Griffin III"

browns_df2017 = yearly_data(df=browns_df2017, year=2017, 
                        HC='Hue Jackson', 
                        OC='Hue Jackson', 
                        DC='Gregg Williams', 
                        QB='DeShone Kizer', 
                        QB2='Kevin Hogan', QB2_range=6)

browns_df2018 = yearly_data(df=browns_df2018, year=2018, 
                        HC='Hue Jackson', HC2='Gregg Williams', HC2_range=range(9,18,1), 
                        OC='Todd Haley', 
                        DC='Gregg Williams', 
                        QB='Baker Mayfield', 
                        QB2='Tyrod Taylor', QB2_range=range(1,4,1))
browns_df2018.loc[browns_df2018['Week'] == range(9,18,1), 'OC'] = "Freddie Kitchens"

browns_df2019 = yearly_data(df=browns_df2019, year=2019, 
                        HC='Freddie Kitchens', 
                        OC='Todd Monken', 
                        DC='Steve Wilks', 
                        QB='Baker Mayfield')

# Make master dataframe for team
browns_df = browns_df2019.append(browns_df2018, ignore_index=True)
browns_df = browns_df.append(browns_df2017, ignore_index=True)
browns_df = browns_df.append(browns_df2016, ignore_index=True)
browns_df = browns_df.append(browns_df2015, ignore_index=True)
browns_df = browns_df.append(browns_df2014, ignore_index=True)
browns_df = browns_df.append(browns_df2013, ignore_index=True)
browns_df = browns_df.append(browns_df2012, ignore_index=True)
browns_df = browns_df.append(browns_df2011, ignore_index=True)
browns_df = browns_df.append(browns_df2010, ignore_index=True)

browns_df['Base_Team'] = 'Browns'
browns_df = browns_df.sort_values(by=['year', 'Week'], ascending=[False, True])

browns_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Tennessee Titans,2019,Browns,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Freddie Kitchens,Todd Monken,Steve Wilks,Baker Mayfield
1,2,Mon,1.0,0.0,0.0,New York Jets,2019,Browns,13.0,43.0,...,0.0,216.0,123.0,346.0,339.0,0,Freddie Kitchens,Todd Monken,Steve Wilks,Baker Mayfield
2,3,Sun,0.0,1.0,1.0,Los Angeles Rams,2019,Browns,36.0,46.0,...,1.0,385.0,216.0,721.0,601.0,0,Freddie Kitchens,Todd Monken,Steve Wilks,Baker Mayfield
3,4,Sun,1.0,1.0,0.0,Baltimore Ravens,2019,Browns,49.0,66.0,...,4.0,639.0,306.0,991.0,945.0,0,Freddie Kitchens,Todd Monken,Steve Wilks,Baker Mayfield
4,5,Mon,0.0,2.0,0.0,San Francisco 49ers,2019,Browns,89.0,91.0,...,7.0,861.0,479.0,1521.0,1340.0,0,Freddie Kitchens,Todd Monken,Steve Wilks,Baker Mayfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,4.0,0.0,Miami Dolphins,2010,Browns,216.0,229.0,...,23.0,2609.0,1315.0,3413.0,3924.0,0,Eric Mangini,Brian Daboll,Rob Ryan,Jake Delhomme
156,14,Sun,0.0,5.0,0.0,Buffalo Bills,2010,Browns,229.0,239.0,...,26.0,2776.0,1429.0,3665.0,4205.0,0,Eric Mangini,Brian Daboll,Rob Ryan,Jake Delhomme
157,15,Sun,0.0,5.0,0.0,Cincinnati Bengals,2010,Browns,235.0,252.0,...,27.0,2907.0,1621.0,3852.0,4528.0,0,Eric Mangini,Brian Daboll,Rob Ryan,Jake Delhomme
158,16,Sun,0.0,5.0,1.0,Baltimore Ravens,2010,Browns,252.0,271.0,...,27.0,3116.0,1809.0,4130.0,4925.0,0,Eric Mangini,Brian Daboll,Rob Ryan,Jake Delhomme


In [413]:
browns_df.to_csv('Resources/indv_team_csvs/browns.csv')

### (25) Colts Data, 2010-2019

In [414]:
# Data Source: https://www.pro-football-reference.com/teams/clt/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

colts_df2010 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2010.csv')
colts_df2011 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2011.csv')
colts_df2012 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2012.csv')
colts_df2013 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2013.csv')
colts_df2014 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2014.csv')
colts_df2015 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2015.csv')
colts_df2016 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2016.csv')
colts_df2017 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2017.csv')
colts_df2018 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2018.csv')
colts_df2019 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2019.csv')

colts_df2010 = yearly_data(df=colts_df2010, year=2010, 
                        HC='Jim Caldwell', 
                        OC='Clyde Christensen', 
                        DC='Larry Coyer', 
                        QB='Peyton Manning')

colts_df2011 = yearly_data(df=colts_df2011, year=2011, 
                        HC='Jim Caldwell', 
                        OC='Clyde Christensen', 
                        DC='Larry Coyer', 
                        QB='Curtis Painter',
                        QB2='Dan Orlovsky', QB2_range=range(13,18,1))
colts_df2011.loc[colts_df2011['Week'] == range(1,4,1), 'QB'] = "Kerry Collins"

colts_df2012 = yearly_data(df=colts_df2012, year=2012, 
                        HC='Chuck Pagano', 
                        OC='Bruce Arians', 
                        DC='Greg Manusky',
                        QB='Andrew Luck')

colts_df2013 = yearly_data(df=colts_df2013, year=2013, 
                        HC='Chuck Pagano', 
                        OC='Pep Hamilton', 
                        DC='Greg Manusky', 
                        QB='Andrew Luck')

colts_df2014 = yearly_data(df=colts_df2014, year=2014, 
                        HC='Chuck Pagano', 
                        OC='Pep Hamilton', 
                        DC='Greg Manusky', 
                        QB='Andrew Luck')

colts_df2015 = yearly_data(df=colts_df2015, year=2015, 
                        HC='Chuck Pagano', 
                        OC='Pep Hamilton', 
                        DC='Greg Manusky', 
                        QB='Matt Hasselbeck',
                        QB2='Andrew Luck', QB2_range=range(1,4,1))
colts_df2015.loc[colts_df2015['Week'] == range(9,18,1), 'OC'] = "Rob Chudzinski"
colts_df2015.loc[colts_df2015['Week'] == 17, 'QB'] = "Josh Freeman"
colts_df2015.loc[colts_df2015['Week'] == range(6,10,1), 'QB'] = "Andrew Luck"

colts_df2016 = yearly_data(df=colts_df2016, year=2016, 
                        HC='Chuck Pagano',
                        OC='Rob Chudzinski', 
                        DC="Ted Monachino", 
                        QB='Andrew Luck', 
                        QB2='Scott Tolzien', QB2_range=12)

colts_df2017 = yearly_data(df=colts_df2017, year=2017, 
                        HC='Chuck Pagano',
                        OC='Rob Chudzinski', 
                        DC="Ted Monachino", 
                        QB='Jacoby Brissett', 
                        QB2='Scott Tolzien', QB2_range=1)

colts_df2018 = yearly_data(df=colts_df2018, year=2018, 
                        HC='Frank Reich', 
                        OC='Nick Sirianni', 
                        DC='Matt Eberflus', 
                        QB='Andrew Luck')

colts_df2019 = yearly_data(df=colts_df2019, year=2019, 
                        HC='Frank Reich', 
                        OC='Nick Sirianni', 
                        DC='Matt Eberflus', 
                        QB='Jacoby Brissett', 
                        QB2='Brian Hoyer', QB2_range=10)

# Make master dataframe for team
colts_df = colts_df2019.append(colts_df2018, ignore_index=True)
colts_df = colts_df.append(colts_df2017, ignore_index=True)
colts_df = colts_df.append(colts_df2016, ignore_index=True)
colts_df = colts_df.append(colts_df2015, ignore_index=True)
colts_df = colts_df.append(colts_df2014, ignore_index=True)
colts_df = colts_df.append(colts_df2013, ignore_index=True)
colts_df = colts_df.append(colts_df2012, ignore_index=True)
colts_df = colts_df.append(colts_df2011, ignore_index=True)
colts_df = colts_df.append(colts_df2010, ignore_index=True)

colts_df['Base_Team'] = 'Colts'
colts_df = colts_df.sort_values(by=['year', 'Week'], ascending=[False, True])

colts_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Los Angeles Chargers,2019,Colts,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Jacoby Brissett
1,2,Sun,1.0,0.0,0.0,Tennessee Titans,2019,Colts,24.0,30.0,...,2.0,310.0,125.0,376.0,435.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Jacoby Brissett
2,3,Sun,1.0,1.0,1.0,Atlanta Falcons,2019,Colts,43.0,47.0,...,2.0,429.0,249.0,664.0,678.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Jacoby Brissett
3,4,Sun,0.0,2.0,1.0,Oakland Raiders,2019,Colts,70.0,71.0,...,3.0,733.0,342.0,1043.0,1075.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Jacoby Brissett
4,5,Sun,1.0,2.0,0.0,Kansas City Chiefs,2019,Colts,94.0,102.0,...,4.0,922.0,530.0,1389.0,1452.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Jacoby Brissett
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,6.0,1.0,Dallas Cowboys,2010,Colts,282.0,252.0,...,16.0,2249.0,1496.0,4184.0,3745.0,0,Jim Caldwell,Clyde Christensen,Larry Coyer,Peyton Manning
156,14,Thu,1.0,6.0,0.0,Tennessee Titans,2010,Colts,317.0,290.0,...,16.0,2400.0,1713.0,4589.0,4113.0,0,Jim Caldwell,Clyde Christensen,Larry Coyer,Peyton Manning
157,15,Sun,1.0,7.0,1.0,Jacksonville Jaguars,2010,Colts,347.0,318.0,...,18.0,2644.0,1834.0,4988.0,4478.0,0,Jim Caldwell,Clyde Christensen,Larry Coyer,Peyton Manning
158,16,Sun,1.0,8.0,0.0,Oakland Raiders,2010,Colts,381.0,342.0,...,20.0,2933.0,1901.0,5364.0,4834.0,0,Jim Caldwell,Clyde Christensen,Larry Coyer,Peyton Manning


In [415]:
colts_df.to_csv('Resources/indv_team_csvs/colts.csv')

### Broncos Data, 2010-2019

In [416]:
# Data Source: https://www.pro-football-reference.com/teams/den/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

broncos_df2010 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2010.csv')
broncos_df2011 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2011.csv')
broncos_df2012 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2012.csv')
broncos_df2013 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2013.csv')
broncos_df2014 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2014.csv')
broncos_df2015 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2015.csv')
broncos_df2016 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2016.csv')
broncos_df2017 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2017.csv')
broncos_df2018 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2018.csv')
broncos_df2019 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2019.csv')

broncos_df2010 = yearly_data(df=broncos_df2010, year=2010, 
                        HC='Josh McDaniels', HC2='Eric Studesville', HC2_range=range(14,18,1), 
                        OC='Mike McCoy', 
                        DC='Don Martindale', 
                        QB='Kyle Orton', 
                        QB2='Tim Tebow', QB2_range=range(15,18,1))

broncos_df2011 = yearly_data(df=broncos_df2011, year=2011, 
                        HC='John Fox', 
                        OC='Mike McCoy', 
                        DC='Dennis Allen', 
                        QB='Tim Tebow', 
                        QB2='Kyle Orton', QB2_range=range(1,6,1))

broncos_df2012 = yearly_data(df=broncos_df2012, year=2012, 
                        HC='John Fox', 
                        OC='Mike McCoy', 
                        DC='Jack Del Rio',
                        QB='Peyton Manning')

broncos_df2013 = yearly_data(df=broncos_df2013, year=2013, 
                        HC='John Fox', 
                        OC='Adam Gase', 
                        DC='Jack Del Rio', 
                        QB='Peyton Manning')

broncos_df2014 = yearly_data(df=broncos_df2014, year=2014, 
                        HC='John Fox', 
                        OC='Adam Gase', 
                        DC='Jack Del Rio', 
                        QB='Peyton Manning')

broncos_df2015 = yearly_data(df=broncos_df2015, year=2015, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Wade Phillips', 
                        QB='Peyton Manning',
                        QB2='Brock Osweiler', QB2_range=range(11,18,1))

broncos_df2016 = yearly_data(df=broncos_df2016, year=2016, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Wade Phillips', 
                        QB='Trevor Siemian',
                        QB2='Paxton Lynch', QB2_range=5)
broncos_df2016.loc[broncos_df2016['Week'] == 13, 'QB'] = "Paxton Lynch"

broncos_df2017 = yearly_data(df=broncos_df2017, year=2017, 
                        HC='Vance Joseph', 
                        OC='Mike McCoy', 
                        DC='Joe Woods', 
                        QB='Trevor Siemian',
                        QB2='Brock Osweiler', QB2_range=range(9,12,1))
broncos_df2017.loc[broncos_df2017['Week'] == 12, 'QB'] = "Paxton Lynch"
broncos_df2017.loc[broncos_df2017['Week'] == 16, 'QB'] = "Brock Osweiler"
broncos_df2017.loc[broncos_df2017['Week'] == 17, 'QB'] = "Paxton Lynch"
broncos_df2017.loc[broncos_df2017['Week'] == range(12,18,1), 'OC'] = "Bill Musgrave"

broncos_df2018 = yearly_data(df=broncos_df2018, year=2018, 
                        HC='Vance Joseph', 
                        OC='Bill Musgrave', 
                        DC='Joe Woods', 
                        QB='Case Keenum')

broncos_df2019 = yearly_data(df=broncos_df2019, year=2019, 
                        HC='Vic Fangio', 
                        OC='Rich Scangarello', 
                        DC='Ed Donatell', 
                        QB='Joe Flacco',
                        QB2='Drew Lock', QB2_range=range(13,18,1))
broncos_df2019.loc[broncos_df2019['Week'] == range(9,13,1), 'QB'] = "Brandon Allen"

# Make master dataframe for team
broncos_df = broncos_df2019.append(broncos_df2018, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2017, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2016, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2015, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2014, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2013, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2012, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2011, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2010, ignore_index=True)

broncos_df['Base_Team'] = 'Broncos'
broncos_df = broncos_df.sort_values(by=['year', 'Week'], ascending=[False, True])

broncos_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Mon,0.0,0.0,0.0,Oakland Raiders,2019,Broncos,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Vic Fangio,Rich Scangarello,Ed Donatell,Joe Flacco
1,2,Sun,0.0,0.0,1.0,Chicago Bears,2019,Broncos,16.0,24.0,...,0.0,259.0,98.0,344.0,357.0,0,Vic Fangio,Rich Scangarello,Ed Donatell,Joe Flacco
2,3,Sun,0.0,0.0,0.0,Green Bay Packers,2019,Broncos,30.0,40.0,...,0.0,379.0,251.0,716.0,630.0,0,Vic Fangio,Rich Scangarello,Ed Donatell,Joe Flacco
3,4,Sun,0.0,0.0,1.0,Jacksonville Jaguars,2019,Broncos,46.0,67.0,...,0.0,614.0,328.0,1026.0,942.0,0,Vic Fangio,Rich Scangarello,Ed Donatell,Joe Flacco
4,5,Sun,1.0,0.0,0.0,Los Angeles Chargers,2019,Broncos,70.0,93.0,...,0.0,800.0,597.0,1397.0,1397.0,0,Vic Fangio,Rich Scangarello,Ed Donatell,Joe Flacco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,3.0,0.0,Kansas City Chiefs,2010,Broncos,250.0,323.0,...,11.0,2629.0,1558.0,4044.0,4187.0,0,Josh McDaniels,Mike McCoy,Don Martindale,Kyle Orton
156,14,Sun,0.0,3.0,0.0,Arizona Cardinals,2010,Broncos,256.0,333.0,...,12.0,2803.0,1743.0,4291.0,4546.0,0,Josh McDaniels,Mike McCoy,Don Martindale,Kyle Orton
157,15,Sun,0.0,3.0,0.0,Oakland Raiders,2010,Broncos,269.0,376.0,...,13.0,2949.0,1954.0,4579.0,4903.0,0,Josh McDaniels,Mike McCoy,Don Martindale,Kyle Orton
158,16,Sun,1.0,3.0,1.0,Houston Texans,2010,Broncos,292.0,415.0,...,16.0,3187.0,2218.0,4814.0,5405.0,0,Josh McDaniels,Mike McCoy,Don Martindale,Kyle Orton


In [417]:
broncos_df.to_csv('Resources/indv_team_csvs/broncos.csv')

### Texans Data, 2010-2019

In [418]:
# Data Source: https://www.pro-football-reference.com/teams/htx/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

texans_df2010 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2010.csv')
texans_df2011 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2011.csv')
texans_df2012 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2012.csv')
texans_df2013 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2013.csv')
texans_df2014 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2014.csv')
texans_df2015 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2015.csv')
texans_df2016 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2016.csv')
texans_df2017 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2017.csv')
texans_df2018 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2018.csv')
texans_df2019 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2019.csv')

texans_df2010 = yearly_data(df=texans_df2010, year=2010, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Frank Bush', 
                        QB='Matt Schaub')

texans_df2011 = yearly_data(df=texans_df2011, year=2011, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Wade Phillips', 
                        QB='Matt Schaub', 
                        QB2='T. J. Yates', QB2_range=range(13,18,1))
texans_df2011.loc[texans_df2011['Week'] == 1, 'QB'] = "Matt Leinart"

texans_df2012 = yearly_data(df=texans_df2012, year=2012, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Wade Phillips',
                        QB='Matt Schaub')

texans_df2013 = yearly_data(df=texans_df2013, year=2013, 
                        HC='Gary Kubiak', HC2='Wade Phillips', HC2_range=range(15,18,1), 
                        OC='Rick Dennison', 
                        DC='Wade Phillips', 
                        QB='Matt Schaub', 
                        QB2='Case Keenum', QB2_range=range(7,17,1))

texans_df2014 = yearly_data(df=texans_df2014, year=2014, 
                        HC="Bill O'Brien", 
                        OC="Bill O'Brien", 
                        DC='Romeo Crennel', 
                        QB='Ryan Fitzpatrick', 
                        QB2='Case Keenum', QB2_range=range(16,18,1))
texans_df2014.loc[texans_df2014['Week'] == range(11,13,1), 'QB'] = "Ryan Mallett"

texans_df2015 = yearly_data(df=texans_df2015, year=2015, 
                        HC="Bill O'Brien", 
                        OC='George Godsey', 
                        DC='Romeo Crennel', 
                        QB='Brian Hoyer', 
                        QB2='Ryan Mallett', QB2_range=range(2,6,1))
texans_df2015.loc[texans_df2015['Week'] == 11, 'QB'] = "T. J. Yates"
texans_df2015.loc[texans_df2015['Week'] == 15, 'QB'] = "T. J. Yates"
texans_df2015.loc[texans_df2015['Week'] == 16, 'QB'] = "Brandon Weeden"

texans_df2016 = yearly_data(df=texans_df2016, year=2016, 
                        HC="Bill O'Brien",
                        OC='George Godsey', 
                        DC='Romeo Crennel', 
                        QB='Brock Osweiler', 
                        QB2='Tom Savage', QB2_range=range(16,18,1))

texans_df2017 = yearly_data(df=texans_df2017, year=2017, 
                        HC="Bill O'Brien", 
                        OC="Bill O'Brien", 
                        DC='Mike Vrabel', 
                        QB='Tom Savage', 
                        QB2='Deshaun Watson', QB2_range=range(2,9,1))
texans_df2017.loc[texans_df2017['Week'] == range(15,18,1), 'QB'] = "T. J. Yates"

texans_df2018 = yearly_data(df=texans_df2018, year=2018, 
                        HC="Bill O'Brien", 
                        OC="Bill O'Brien", 
                        DC='Romeo Crennel', 
                        QB='Deshaun Watson')

texans_df2019 = yearly_data(df=texans_df2019, year=2019, 
                        HC="Bill O'Brien", 
                        OC='Tim Kelly', 
                        DC='Romeo Crennel', 
                        QB='Deshaun Watson', 
                        QB2='A. J. McCarron', QB2_range=17)

# Make master dataframe for team
texans_df = texans_df2019.append(texans_df2018, ignore_index=True)
texans_df = texans_df.append(texans_df2017, ignore_index=True)
texans_df = texans_df.append(texans_df2016, ignore_index=True)
texans_df = texans_df.append(texans_df2015, ignore_index=True)
texans_df = texans_df.append(texans_df2014, ignore_index=True)
texans_df = texans_df.append(texans_df2013, ignore_index=True)
texans_df = texans_df.append(texans_df2012, ignore_index=True)
texans_df = texans_df.append(texans_df2011, ignore_index=True)
texans_df = texans_df.append(texans_df2010, ignore_index=True)

texans_df['Base_Team'] = 'Texans'
texans_df = texans_df.sort_values(by=['year', 'Week'], ascending=[False, True])

texans_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Mon,0.0,0.0,0.0,New Orleans Saints,2019,Texans,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Bill O'Brien,Tim Kelly,Romeo Crennel,Deshaun Watson
1,2,Sun,1.0,0.0,1.0,Jacksonville Jaguars,2019,Texans,28.0,30.0,...,1.0,362.0,148.0,414.0,510.0,0,Bill O'Brien,Tim Kelly,Romeo Crennel,Deshaun Watson
2,3,Sun,1.0,1.0,0.0,Los Angeles Chargers,2019,Texans,41.0,42.0,...,2.0,540.0,251.0,677.0,791.0,0,Bill O'Brien,Tim Kelly,Romeo Crennel,Deshaun Watson
3,4,Sun,0.0,2.0,1.0,Carolina Panthers,2019,Texans,68.0,62.0,...,3.0,833.0,324.0,1053.0,1157.0,0,Bill O'Brien,Tim Kelly,Romeo Crennel,Deshaun Watson
4,5,Sun,1.0,2.0,1.0,Atlanta Falcons,2019,Texans,78.0,78.0,...,6.0,1036.0,418.0,1317.0,1454.0,0,Bill O'Brien,Tim Kelly,Romeo Crennel,Deshaun Watson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Thu,0.0,5.0,0.0,Philadelphia Eagles,2010,Texans,264.0,287.0,...,13.0,3148.0,1102.0,4045.0,4250.0,0,Gary Kubiak,Rick Dennison,Frank Bush,Matt Schaub
156,14,Mon,0.0,5.0,1.0,Baltimore Ravens,2010,Texans,288.0,321.0,...,14.0,3449.0,1217.0,4476.0,4666.0,0,Gary Kubiak,Rick Dennison,Frank Bush,Matt Schaub
157,15,Sun,0.0,5.0,0.0,Tennessee Titans,2010,Texans,316.0,355.0,...,14.0,3639.0,1280.0,4965.0,4919.0,0,Gary Kubiak,Rick Dennison,Frank Bush,Matt Schaub
158,16,Sun,0.0,5.0,0.0,Denver Broncos,2010,Texans,333.0,386.0,...,15.0,3851.0,1427.0,5288.0,5278.0,0,Gary Kubiak,Rick Dennison,Frank Bush,Matt Schaub


In [419]:
texans_df.to_csv('Resources/indv_team_csvs/texans.csv')

### Jets Data, 2010-2019

In [420]:
# Data Source: https://www.pro-football-reference.com/teams/nyj/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

jets_df2010 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2010.csv')
jets_df2011 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2011.csv')
jets_df2012 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2012.csv')
jets_df2013 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2013.csv')
jets_df2014 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2014.csv')
jets_df2015 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2015.csv')
jets_df2016 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2016.csv')
jets_df2017 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2017.csv')
jets_df2018 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2018.csv')
jets_df2019 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2019.csv')

jets_df2010 = yearly_data(df=jets_df2010, year=2010, 
                        HC='Rex Ryan', 
                        OC='Brian Schottenheimer', 
                        DC='Mike Pettine', 
                        QB='Mark Sanchez')

jets_df2011 = yearly_data(df=jets_df2011, year=2011, 
                        HC='Rex Ryan', 
                        OC='Brian Schottenheimer', 
                        DC='Mike Pettine', 
                        QB='Mark Sanchez')

jets_df2012 = yearly_data(df=jets_df2012, year=2012, 
                        HC='Rex Ryan', 
                        OC='Tony Sparano', 
                        DC='Mike Pettine',
                        QB='Mark Sanchez', 
                        QB2='Greg McElroy', QB2_range=16)

jets_df2013 = yearly_data(df=jets_df2013, year=2013, 
                        HC='Rex Ryan', 
                        OC='Marty Mornhinweg', 
                        DC='Dennis Thurman', 
                        QB='Geno Smith')

jets_df2014 = yearly_data(df=jets_df2014, year=2014, 
                        HC='Rex Ryan', 
                        OC='Marty Mornhinweg', 
                        DC='Dennis Thurman', 
                        QB='Geno Smith', 
                        QB2='Michael Vick', QB2_range=range(9,13,1))

jets_df2015 = yearly_data(df=jets_df2015, year=2015, 
                        HC='Todd Bowles', 
                        OC='Chan Gailey', 
                        DC='Kacy Rodgers', 
                        QB='Ryan Fitzpatrick')

jets_df2016 = yearly_data(df=jets_df2016, year=2016, 
                        HC='Todd Bowles', 
                        OC='Chan Gailey', 
                        DC='Kacy Rodgers', 
                        QB='Ryan Fitzpatrick', 
                        QB2='Bryce Petty', QB2_range=range(14,17,1))
jets_df2016.loc[jets_df2016['Week'] == 7, 'QB'] = "Geno Smith"
jets_df2016.loc[jets_df2016['Week'] == 10, 'QB'] = "Bryce Petty"

jets_df2017 = yearly_data(df=jets_df2017, year=2017, 
                        HC='Todd Bowles', 
                        OC='John Morton', 
                        DC='Kacy Rodgers', 
                        QB='Josh McCown', 
                        QB2='Bryce Petty', QB2_range=range(15,18,1))

jets_df2018 = yearly_data(df=jets_df2018, year=2018, 
                        HC='Todd Bowles', 
                        OC='Jeremy Bates', 
                        DC='Kacy Rodgers', 
                        QB='Sam Darnold', 
                        QB2='Josh McCown', QB2_range=range(10,14,1))

jets_df2019 = yearly_data(df=jets_df2019, year=2019, 
                        HC='Adam Gase', 
                        OC='Dowell Loggains', 
                        DC='Gregg Williams', 
                        QB='Sam Darnold', 
                        QB2='Luke Falk', QB2_range=range(3,6,2))
jets_df2019.loc[jets_df2019['Week'] == 1, 'QB'] = "Trevor Siemian"

# Make master dataframe for team
jets_df = jets_df2019.append(jets_df2018, ignore_index=True)
jets_df = jets_df.append(jets_df2017, ignore_index=True)
jets_df = jets_df.append(jets_df2016, ignore_index=True)
jets_df = jets_df.append(jets_df2015, ignore_index=True)
jets_df = jets_df.append(jets_df2014, ignore_index=True)
jets_df = jets_df.append(jets_df2013, ignore_index=True)
jets_df = jets_df.append(jets_df2012, ignore_index=True)
jets_df = jets_df.append(jets_df2011, ignore_index=True)
jets_df = jets_df.append(jets_df2010, ignore_index=True)

jets_df['Base_Team'] = 'Jets'
jets_df = jets_df.sort_values(by=['year', 'Week'], ascending=[False, True])

jets_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Buffalo Bills,2019,Jets,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Trevor Siemian
1,2,Mon,0.0,0.0,1.0,Cleveland Browns,2019,Jets,16.0,17.0,...,4.0,242.0,128.0,223.0,370.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
2,3,Sun,0.0,0.0,0.0,New England Patriots,2019,Jets,19.0,40.0,...,5.0,547.0,198.0,485.0,745.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
3,5,Sun,0.0,0.0,0.0,Philadelphia Eagles,2019,Jets,33.0,70.0,...,7.0,860.0,266.0,590.0,1126.0,1,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
4,6,Sun,1.0,0.0,1.0,Dallas Cowboys,2019,Jets,39.0,101.0,...,8.0,1041.0,350.0,718.0,1391.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Mon,0.0,9.0,0.0,New England Patriots,2010,Jets,264.0,187.0,...,20.0,2317.0,949.0,3978.0,3266.0,0,Rex Ryan,Brian Schottenheimer,Mike Pettine,Mark Sanchez
156,14,Sun,0.0,9.0,1.0,Miami Dolphins,2010,Jets,267.0,232.0,...,20.0,2621.0,1050.0,4279.0,3671.0,0,Rex Ryan,Brian Schottenheimer,Mike Pettine,Mark Sanchez
157,15,Sun,1.0,9.0,0.0,Pittsburgh Steelers,2010,Jets,273.0,242.0,...,23.0,2651.0,1151.0,4559.0,3802.0,0,Rex Ryan,Brian Schottenheimer,Mike Pettine,Mark Sanchez
158,16,Sun,0.0,10.0,0.0,Chicago Bears,2010,Jets,295.0,259.0,...,23.0,2883.0,1297.0,4835.0,4180.0,0,Rex Ryan,Brian Schottenheimer,Mike Pettine,Mark Sanchez


In [421]:
jets_df.to_csv('Resources/indv_team_csvs/jets.csv')

### Jaguars Data, 2010-2019

In [422]:
# Data Source: https://www.pro-football-reference.com/teams/jax/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

jaguars_df2010 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2010.csv')
jaguars_df2011 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2011.csv')
jaguars_df2012 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2012.csv')
jaguars_df2013 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2013.csv')
jaguars_df2014 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2014.csv')
jaguars_df2015 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2015.csv')
jaguars_df2016 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2016.csv')
jaguars_df2017 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2017.csv')
jaguars_df2018 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2018.csv')
jaguars_df2019 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2019.csv')

jaguars_df2010 = yearly_data(df=jaguars_df2010, year=2010, 
                        HC='Jack Del Rio', 
                        OC='Dirk Koetter', 
                        DC='Mel Tucker', 
                        QB='David Garrard',
                        QB2='Todd Bouman', QB2_range=7)
jaguars_df2010.loc[jaguars_df2010['Week'] == 17, 'QB'] = "Trent Edwards"

jaguars_df2011 = yearly_data(df=jaguars_df2011, year=2011, 
                        HC='Jack Del Rio', HC2='Mel Tucker', HC2_range=range(13,18,1),
                        OC='Dirk Koetter', 
                        DC='Mel Tucker', 
                        QB='Blaine Gabbert',
                        QB2='Luke McCown', QB2_range=range(1,3,1))

jaguars_df2012 = yearly_data(df=jaguars_df2012, year=2012, 
                        HC='Mike Mularkey', 
                        OC='Bob Bratkowski', 
                        DC='Mel Tucker',
                        QB='Blaine Gabbert',
                        QB2='Chad Henne', QB2_range=range(12,18,1))

jaguars_df2013 = yearly_data(df=jaguars_df2013, year=2013, 
                        HC='Gus Bradley', 
                        OC='Jedd Fisch', 
                        DC='Bob Babich', 
                        QB='Chad Henne',
                        QB2='Blaine Gabbert', QB2_range=range(1,5,3))

jaguars_df2014 = yearly_data(df=jaguars_df2014, year=2014, 
                        HC='Gus Bradley', 
                        OC='Jedd Fisch', 
                        DC='Bob Babich', 
                        QB='Blake Bortles',
                        QB2='Chad Henne', QB2_range=range(1,4,1))

jaguars_df2015 = yearly_data(df=jaguars_df2015, year=2015, 
                        HC='Gus Bradley', 
                        OC='Greg Olson', 
                        DC='Bob Babich', 
                        QB='Blake Bortles')

jaguars_df2016 = yearly_data(df=jaguars_df2016, year=2016, 
                        HC='Gus Bradley', HC2='Doug Marrone', HC2_range=range(16,18,1),
                        OC='Greg Olson', 
                        DC="Todd Wash", 
                        QB='Blake Bortles')
jaguars_df2016.loc[jaguars_df2016['Week'] == range(16,18,1), 'OC'] = "Nathaniel Hackett"

jaguars_df2017 = yearly_data(df=jaguars_df2017, year=2017, 
                        HC='Doug Marrone', 
                        OC='Nathaniel Hackett', 
                        DC='Todd Wash', 
                        QB='Blake Bortles')

jaguars_df2018 = yearly_data(df=jaguars_df2018, year=2018, 
                        HC='Doug Marrone', 
                        OC='Nathaniel Hackett', 
                        DC='Todd Wash', 
                        QB='Blake Bortles',
                        QB2='Cody Kessler', QB2_range=range(13,17,1))

jaguars_df2019 = yearly_data(df=jaguars_df2019, year=2019, 
                        HC='Doug Marrone', 
                        OC='John DeFilippo', 
                        DC='Todd Wash', 
                        QB='Gardner Minshew',
                        QB2='Nick Foles', QB2_range=range(1,3,1))
jaguars_df2019.loc[jaguars_df2019['Week'] == range(12,14,1), 'QB'] = "Nick Foles"

# Make master dataframe for team
jaguars_df = jaguars_df2019.append(jaguars_df2018, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2017, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2016, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2015, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2014, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2013, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2012, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2011, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2010, ignore_index=True)

jaguars_df['Base_Team'] = 'Jaguars'
jaguars_df = jaguars_df.sort_values(by=['year', 'Week'], ascending=[False, True])

jaguars_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Kansas City Chiefs,2019,Jaguars,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Doug Marrone,John DeFilippo,Todd Wash,Gardner Minshew
1,2,Sun,0.0,0.0,0.0,Houston Texans,2019,Jaguars,26.0,40.0,...,0.0,378.0,113.0,428.0,491.0,0,Doug Marrone,John DeFilippo,Todd Wash,Gardner Minshew
2,3,Thu,1.0,0.0,1.0,Tennessee Titans,2019,Jaguars,38.0,53.0,...,0.0,515.0,239.0,709.0,754.0,0,Doug Marrone,John DeFilippo,Todd Wash,Gardner Minshew
3,4,Sun,1.0,1.0,0.0,Denver Broncos,2019,Jaguars,58.0,60.0,...,1.0,764.0,330.0,1001.0,1094.0,0,Doug Marrone,John DeFilippo,Todd Wash,Gardner Minshew
4,5,Sun,0.0,2.0,0.0,Carolina Panthers,2019,Jaguars,84.0,84.0,...,2.0,1067.0,398.0,1456.0,1465.0,0,Doug Marrone,John DeFilippo,Todd Wash,Gardner Minshew
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,1.0,6.0,0.0,Tennessee Titans,2010,Jaguars,240.0,294.0,...,13.0,2872.0,1257.0,3684.0,4129.0,0,Jack Del Rio,Dirk Koetter,Mel Tucker,David Garrard
156,14,Sun,1.0,7.0,1.0,Oakland Raiders,2010,Jaguars,257.0,300.0,...,15.0,3035.0,1314.0,4061.0,4349.0,0,Jack Del Rio,Dirk Koetter,Mel Tucker,David Garrard
157,15,Sun,0.0,8.0,0.0,Indianapolis Colts,2010,Jaguars,295.0,331.0,...,17.0,3358.0,1467.0,4446.0,4825.0,0,Jack Del Rio,Dirk Koetter,Mel Tucker,David Garrard
158,16,Sun,0.0,8.0,1.0,Washington Redskins,2010,Jaguars,319.0,365.0,...,17.0,3579.0,1622.0,4802.0,5201.0,0,Jack Del Rio,Dirk Koetter,Mel Tucker,David Garrard


In [423]:
jaguars_df.to_csv('Resources/indv_team_csvs/jaguars.csv')

### (30) Dolphins Data, 2010-2019

In [424]:
# Data Source: https://www.pro-football-reference.com/teams/mia/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

dolphins_df2010 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2010.csv')
dolphins_df2011 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2011.csv')
dolphins_df2012 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2012.csv')
dolphins_df2013 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2013.csv')
dolphins_df2014 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2014.csv')
dolphins_df2015 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2015.csv')
dolphins_df2016 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2016.csv')
dolphins_df2017 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2017.csv')
dolphins_df2018 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2018.csv')
dolphins_df2019 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2019.csv')

dolphins_df2010 = yearly_data(df=dolphins_df2010, year=2010, 
                        HC='Tony Sparano', 
                        OC='Dan Henning', 
                        DC='Mike Nolan', 
                        QB='Chad Henne',
                        QB2='Tyler Thigpen', QB2_range=11)
dolphins_df2010.loc[dolphins_df2010['Week'] == 10, 'QB'] = "Chad Pennington"

dolphins_df2011 = yearly_data(df=dolphins_df2011, year=2011, 
                        HC='Tony Sparano', 
                        OC='Brian Daboll', 
                        DC='Mike Nolan', 
                        QB='Chad Henne',
                        QB2='Matt Moore', QB2_range=range(1,5,1))

dolphins_df2012 = yearly_data(df=dolphins_df2012, year=2012, 
                        HC='Joe Philbin', 
                        OC='Mike Sherman', 
                        DC='Kevin Coyle',
                        QB='Ryan Tannehill')

dolphins_df2013 = yearly_data(df=dolphins_df2013, year=2013, 
                        HC='Joe Philbin', 
                        OC='Mike Sherman', 
                        DC='Kevin Coyle', 
                        QB='Ryan Tannehill')

dolphins_df2014 = yearly_data(df=dolphins_df2014, year=2014, 
                        HC='Joe Philbin', 
                        OC='Bill Lazor', 
                        DC='Kevin Coyle', 
                        QB='Ryan Tannehill')

dolphins_df2015 = yearly_data(df=dolphins_df2015, year=2015, 
                        HC='Joe Philbin', HC2='Dan Campbell', HC2_range=range(6,18,1), 
                        OC='Bill Lazor', 
                        DC='Kevin Coyle', 
                        QB='Ryan Tannehill')
dolphins_df2015.loc[dolphins_df2015['Week'] == range(6,18,1), 'OC'] = "Zac Taylor"
dolphins_df2015.loc[dolphins_df2015['Week'] == range(6,18,1), 'DC'] = "Lou Anarumo"

dolphins_df2016 = yearly_data(df=dolphins_df2016, year=2016, 
                        HC='Adam Gase',
                        OC='Clyde Christensen', 
                        DC="Vance Joseph", 
                        QB='Ryan Tannehill',
                        QB2='Matt Moore', QB2_range=range(15,18,1))

dolphins_df2017 = yearly_data(df=dolphins_df2017, year=2017, 
                        HC='Adam Gase',
                        OC='Clyde Christensen', 
                        DC='Matt Burke', 
                        QB='Jay Cutler',
                        QB2='Matt Moore', QB2_range=range(8,13,4))

dolphins_df2018 = yearly_data(df=dolphins_df2018, year=2018, 
                        HC='Adam Gase',
                        OC='Dowell Loggains', 
                        DC='Matt Burke', 
                        QB='Ryan Tannehill',
                        QB2='Brock Osweiler', QB2_range=range(6,12,1))

dolphins_df2019 = yearly_data(df=dolphins_df2019, year=2019, 
                        HC='Brian Flores', 
                        OC="Chad O'Shea", 
                        DC='Patrick Graham', 
                        QB='Ryan Fitzpatrick',
                        QB2='Josh Rosen', QB2_range=range(3,7,1))

# Make master dataframe for team
dolphins_df = dolphins_df2019.append(dolphins_df2018, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2017, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2016, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2015, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2014, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2013, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2012, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2011, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2010, ignore_index=True)

dolphins_df['Base_Team'] = 'Dolphins'
dolphins_df = dolphins_df.sort_values(by=['year', 'Week'], ascending=[False, True])

dolphins_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Baltimore Ravens,2019,Dolphins,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Brian Flores,Chad O'Shea,Patrick Graham,Ryan Fitzpatrick
1,2,Sun,0.0,0.0,1.0,New England Patriots,2019,Dolphins,10.0,59.0,...,0.0,378.0,265.0,200.0,643.0,0,Brian Flores,Chad O'Shea,Patrick Graham,Ryan Fitzpatrick
2,3,Sun,0.0,0.0,0.0,Dallas Cowboys,2019,Dolphins,10.0,102.0,...,1.0,633.0,389.0,384.0,1022.0,0,Brian Flores,Chad O'Shea,Patrick Graham,Ryan Fitzpatrick
3,4,Sun,0.0,0.0,1.0,Los Angeles Chargers,2019,Dolphins,16.0,133.0,...,2.0,874.0,624.0,667.0,1498.0,0,Brian Flores,Chad O'Shea,Patrick Graham,Ryan Fitzpatrick
4,6,Sun,0.0,0.0,1.0,Washington Redskins,2019,Dolphins,26.0,163.0,...,2.0,1185.0,703.0,900.0,1888.0,1,Brian Flores,Chad O'Shea,Patrick Graham,Ryan Fitzpatrick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,6.0,1.0,Cleveland Browns,2010,Dolphins,205.0,225.0,...,15.0,2215.0,1143.0,3757.0,3358.0,0,Tony Sparano,Dan Henning,Mike Nolan,Chad Henne
156,14,Sun,1.0,6.0,0.0,New York Jets,2010,Dolphins,215.0,238.0,...,15.0,2415.0,1195.0,4038.0,3610.0,0,Tony Sparano,Dan Henning,Mike Nolan,Chad Henne
157,15,Sun,0.0,7.0,1.0,Buffalo Bills,2010,Dolphins,225.0,244.0,...,17.0,2608.0,1282.0,4169.0,3890.0,0,Tony Sparano,Dan Henning,Mike Nolan,Chad Henne
158,16,Sun,0.0,7.0,1.0,Detroit Lions,2010,Dolphins,239.0,261.0,...,18.0,2819.0,1353.0,4495.0,4172.0,0,Tony Sparano,Dan Henning,Mike Nolan,Chad Henne


In [425]:
dolphins_df.to_csv('Resources/indv_team_csvs/dolphins.csv')

### Patriots Data, 2010-2019

In [426]:
# Data Source: https://www.pro-football-reference.com/teams/nwe/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

pats_df2010 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2010.csv')
pats_df2011 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2011.csv')
pats_df2012 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2012.csv')
pats_df2013 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2013.csv')
pats_df2014 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2014.csv')
pats_df2015 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2015.csv')
pats_df2016 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2016.csv')
pats_df2017 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2017.csv')
pats_df2018 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2018.csv')
pats_df2019 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2019.csv')

pats_df2010 = yearly_data(df=pats_df2010, year=2010, 
                        HC='Bill Belichick', 
                        OC='Bill Belichick', 
                        DC='Bill Belichick', 
                        QB='Tom Brady')

pats_df2011 = yearly_data(df=pats_df2011, year=2011, 
                        HC='Bill Belichick', 
                        OC="Bill O'Brien", 
                        DC='Bill Belichick', 
                        QB='Tom Brady')

pats_df2012 = yearly_data(df=pats_df2012, year=2012, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia',
                        QB='Tom Brady')

pats_df2013 = yearly_data(df=pats_df2013, year=2013, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady')

pats_df2014 = yearly_data(df=pats_df2014, year=2014, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady')

pats_df2015 = yearly_data(df=pats_df2015, year=2015, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady')

pats_df2016 = yearly_data(df=pats_df2016, year=2016, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady', 
                        QB2='Jimmy Garoppolo', QB2_range=range(1,3,1))
pats_df2016.loc[pats_df2016['Week'] == range(3,5,1), 'QB'] = "Jacoby Brissett"

pats_df2017 = yearly_data(df=pats_df2017, year=2017, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady')

pats_df2018 = yearly_data(df=pats_df2018, year=2018, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Bill Belichick', 
                        QB='Tom Brady')

pats_df2019 = yearly_data(df=pats_df2019, year=2019, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Bill Belichick', 
                        QB='Tom Brady')

# Make master dataframe for team
pats_df = pats_df2019.append(pats_df2018, ignore_index=True)
pats_df = pats_df.append(pats_df2017, ignore_index=True)
pats_df = pats_df.append(pats_df2016, ignore_index=True)
pats_df = pats_df.append(pats_df2015, ignore_index=True)
pats_df = pats_df.append(pats_df2014, ignore_index=True)
pats_df = pats_df.append(pats_df2013, ignore_index=True)
pats_df = pats_df.append(pats_df2012, ignore_index=True)
pats_df = pats_df.append(pats_df2011, ignore_index=True)
pats_df = pats_df.append(pats_df2010, ignore_index=True)

pats_df['Base_Team'] = 'Patriots'
pats_df = pats_df.sort_values(by=['year', 'Week'], ascending=[False, True])

pats_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Pittsburgh Steelers,2019,Patriots,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Tom Brady
1,2,Sun,1.0,1.0,0.0,Miami Dolphins,2019,Patriots,33.0,3.0,...,1.0,276.0,32.0,465.0,308.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Tom Brady
2,3,Sun,1.0,2.0,1.0,New York Jets,2019,Patriots,76.0,3.0,...,5.0,418.0,74.0,844.0,492.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Tom Brady
3,4,Sun,1.0,3.0,0.0,Buffalo Bills,2019,Patriots,106.0,17.0,...,6.0,487.0,110.0,1225.0,597.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Tom Brady
4,5,Sun,1.0,4.0,0.0,Washington Redskins,2019,Patriots,122.0,27.0,...,10.0,727.0,245.0,1449.0,972.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Tom Brady
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Mon,1.0,9.0,1.0,New York Jets,2010,Patriots,334.0,266.0,...,20.0,3173.0,1217.0,3841.0,4390.0,0,Bill Belichick,Bill Belichick,Bill Belichick,Tom Brady
156,14,Sun,1.0,10.0,0.0,Chicago Bears,2010,Patriots,379.0,269.0,...,23.0,3322.0,1369.0,4246.0,4691.0,0,Bill Belichick,Bill Belichick,Bill Belichick,Tom Brady
157,15,Sun,1.0,11.0,1.0,Green Bay Packers,2010,Patriots,415.0,276.0,...,27.0,3460.0,1416.0,4721.0,4876.0,0,Bill Belichick,Bill Belichick,Bill Belichick,Tom Brady
158,16,Sun,1.0,12.0,0.0,Buffalo Bills,2010,Patriots,446.0,303.0,...,29.0,3686.0,1559.0,4970.0,5245.0,0,Bill Belichick,Bill Belichick,Bill Belichick,Tom Brady


In [427]:
pats_df.to_csv('Resources/indv_team_csvs/pats.csv')

### Bills Data, 2010-2019

In [428]:
# Data Source: https://www.pro-football-reference.com/teams/buf/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

bills_df2010 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2010.csv')
bills_df2011 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2011.csv')
bills_df2012 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2012.csv')
bills_df2013 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2013.csv')
bills_df2014 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2014.csv')
bills_df2015 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2015.csv')
bills_df2016 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2016.csv')
bills_df2017 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2017.csv')
bills_df2018 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2018.csv')
bills_df2019 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2019.csv')

bills_df2010 = yearly_data(df=bills_df2010, year=2010, 
                        HC='Chan Gailey', 
                        OC='Curtis Modkins', 
                        DC='George Edwards', 
                        QB='Ryan Fitzpatrick', 
                        QB2='Trent Edwards', QB2_range=range(1,3,1))
bills_df2010.loc[bills_df2010['Week'] == 17, 'QB'] = "Brian Brohm"

bills_df2011 = yearly_data(df=bills_df2011, year=2011, 
                        HC='Chan Gailey', 
                        OC='Curtis Modkins', 
                        DC='George Edwards', 
                        QB='Ryan Fitzpatrick')

bills_df2012 = yearly_data(df=bills_df2012, year=2012, 
                        HC='Chan Gailey', 
                        OC='Curtis Modkins', 
                        DC='Dave Wannstedt',
                        QB='Ryan Fitzpatrick')

bills_df2013 = yearly_data(df=bills_df2013, year=2013, 
                        HC='Doug Marrone', 
                        OC='Nathaniel Hackett', 
                        DC='Mike Pettine', 
                        QB='EJ Manuel', 
                        QB2='Thad Lewis', QB2_range=range(6,9,1))
bills_df2013.loc[bills_df2013['Week'] == 9, 'QB'] = "Jeff Tuel"
bills_df2013.loc[bills_df2013['Week'] == range(16,18,1), 'QB'] = "Thad Lewis"

bills_df2014 = yearly_data(df=bills_df2014, year=2014, 
                        HC='Doug Marrone', 
                        OC='Nathaniel Hackett', 
                        DC='Jim Schwartz', 
                        QB='Kyle Orton', 
                        QB2='EJ Manuel', QB2_range=range(1,5,1))

bills_df2015 = yearly_data(df=bills_df2015, year=2015, 
                        HC='Rex Ryan', 
                        OC='Greg Roman', 
                        DC='Dennis Thurman', 
                        QB='Tyrod Taylor', 
                        QB2='EJ Manuel', QB2_range=range(6,8,1))

bills_df2016 = yearly_data(df=bills_df2016, year=2016, 
                        HC='Rex Ryan', HC2='Anthony Lynn', HC2_range=17,
                        OC='Greg Roman', 
                        DC="Dennis Thurman", 
                        QB='Tyrod Taylor', 
                        QB2='EJ Manuel', QB2_range=17)
bills_df2016.loc[bills_df2016['Week'] == range(3,18,1), 'OC'] = "Anthony Lynn"

bills_df2017 = yearly_data(df=bills_df2017, year=2017, 
                        HC='Sean McDermott', 
                        OC='Rick Dennison', 
                        DC='Leslie Frazier', 
                        QB='Tyrod Taylor', 
                        QB2='Nathan Peterman', QB2_range=11)
bills_df2017.loc[bills_df2017['Week'] == 14, 'QB'] = "Nathan Peterman"

bills_df2018 = yearly_data(df=bills_df2018, year=2018, 
                        HC='Sean McDermott', 
                        OC='Brian Daboll', 
                        DC='Leslie Frazier', 
                        QB='Josh Allen', 
                        QB2='Nathan Peterman', QB2_range=1)
bills_df2018.loc[bills_df2018['Week'] == 9, 'QB'] = "Nathan Peterman"
bills_df2018.loc[bills_df2018['Week'] == range(7,9,1), 'QB'] = "Derek Anderson"
bills_df2018.loc[bills_df2018['Week'] == 10, 'QB'] = "Matt Barkley"

bills_df2019 = yearly_data(df=bills_df2019, year=2019, 
                        HC='Sean McDermott', 
                        OC='Brian Daboll', 
                        DC='Leslie Frazier', 
                        QB='Josh Allen')

# Make master dataframe for team
bills_df = bills_df2019.append(bills_df2018, ignore_index=True)
bills_df = bills_df.append(bills_df2017, ignore_index=True)
bills_df = bills_df.append(bills_df2016, ignore_index=True)
bills_df = bills_df.append(bills_df2015, ignore_index=True)
bills_df = bills_df.append(bills_df2014, ignore_index=True)
bills_df = bills_df.append(bills_df2013, ignore_index=True)
bills_df = bills_df.append(bills_df2012, ignore_index=True)
bills_df = bills_df.append(bills_df2011, ignore_index=True)
bills_df = bills_df.append(bills_df2010, ignore_index=True)

bills_df['Base_Team'] = 'Bills'
bills_df = bills_df.sort_values(by=['year', 'Week'], ascending=[False, True])

bills_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,New York Jets,2019,Bills,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
1,2,Sun,1.0,1.0,0.0,New York Giants,2019,Bills,17.0,16.0,...,1.0,155.0,68.0,370.0,223.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
2,3,Sun,1.0,2.0,1.0,Cincinnati Bengals,2019,Bills,45.0,30.0,...,3.0,396.0,197.0,758.0,593.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
3,4,Sun,0.0,3.0,1.0,New England Patriots,2019,Bills,66.0,47.0,...,7.0,636.0,263.0,1174.0,899.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
4,5,Sun,1.0,3.0,0.0,Tennessee Titans,2019,Bills,76.0,63.0,...,8.0,786.0,337.0,1549.0,1123.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,2.0,0.0,Minnesota Vikings,2010,Bills,229.0,295.0,...,12.0,2309.0,1841.0,3503.0,4150.0,0,Chan Gailey,Curtis Modkins,George Edwards,Ryan Fitzpatrick
156,14,Sun,1.0,2.0,1.0,Cleveland Browns,2010,Bills,243.0,333.0,...,16.0,2486.0,2051.0,3742.0,4537.0,0,Chan Gailey,Curtis Modkins,George Edwards,Ryan Fitzpatrick
157,15,Sun,1.0,3.0,0.0,Miami Dolphins,2010,Bills,256.0,339.0,...,19.0,2568.0,2156.0,4065.0,4724.0,0,Chan Gailey,Curtis Modkins,George Edwards,Ryan Fitzpatrick
158,16,Sun,0.0,4.0,1.0,New England Patriots,2010,Bills,273.0,353.0,...,21.0,2829.0,2221.0,4347.0,5050.0,0,Chan Gailey,Curtis Modkins,George Edwards,Ryan Fitzpatrick


In [429]:
bills_df.to_csv('Resources/indv_team_csvs/bills.csv')

### Titans Data, 2010-2019

In [430]:
# Data Source: https://www.pro-football-reference.com/teams/oti/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

titans_df2010 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2010.csv')
titans_df2011 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2011.csv')
titans_df2012 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2012.csv')
titans_df2013 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2013.csv')
titans_df2014 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2014.csv')
titans_df2015 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2015.csv')
titans_df2016 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2016.csv')
titans_df2017 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2017.csv')
titans_df2018 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2018.csv')
titans_df2019 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2019.csv')

titans_df2010 = yearly_data(df=titans_df2010, year=2010, 
                        HC='Jeff Fisher', 
                        OC='Mike Heimerdinger', 
                        DC='Chuck Cecil', 
                        QB='Vince Young', 
                        QB2='Kerry Collins',QB2_range=range(13,18,1))
titans_df2010.loc[titans_df2010['Week'] == 12, 'QB'] = "Rusty Smith"
titans_df2010.loc[titans_df2010['Week'] == 7, 'QB'] = "Rusty Smith"

titans_df2011 = yearly_data(df=titans_df2011, year=2011, 
                        HC='Mike Munchak', 
                        OC='Chris Palmer', 
                        DC='Jerry Gray', 
                        QB='Matt Hasselbeck')

titans_df2012 = yearly_data(df=titans_df2012, year=2012, 
                        HC='Mike Munchak', 
                        OC='Chris Palmer', 
                        DC='Jerry Gray',
                        QB='Jake Locker', 
                        QB2='Matt Hasselbeck',QB2_range=range(5,10,1))

titans_df2013 = yearly_data(df=titans_df2013, year=2013, 
                        HC='Mike Munchak', 
                        OC='Dowell Loggains', 
                        DC='Jerry Gray', 
                        QB='Ryan Fitzpatrick', 
                        QB2='Jake Locker',QB2_range=range(4,11,3))
titans_df2013.loc[titans_df2013['Week'] == range(3,10,6), 'QB'] = "Jake Locker"

titans_df2014 = yearly_data(df=titans_df2014, year=2014, 
                        HC='Ken Whisenhunt', 
                        OC='Jason Michael', 
                        DC='Ray Horton', 
                        QB='Zach Mettenberger',
                        QB2='Jake Locker',QB2_range=range(1,4,1))
titans_df2014.loc[titans_df2014['Week'] == 4, 'QB'] = "Charlie Whitehurst"
titans_df2014.loc[titans_df2014['Week'] == range(6,8,1), 'QB'] = "Charlie Whitehurst"
titans_df2014.loc[titans_df2014['Week'] == range(16,18,1), 'QB'] = "Charlie Whitehurst"
titans_df2014.loc[titans_df2014['Week'] == range(5,16,10), 'QB'] = "Jake Locker"

titans_df2015 = yearly_data(df=titans_df2015, year=2015, 
                        HC='Ken Whisenhunt',HC2='Mike Mularkey', HC2_range=range(9,18,1), 
                        OC='Jason Michael', 
                        DC='Ray Horton', 
                        QB='Marcus Mariota', 
                        QB2='Zach Mettenberger',QB2_range=range(16,18,1))
titans_df2015.loc[titans_df2015['Week'] == range(7,9,1), 'QB'] = "Zach Mettenberger"

titans_df2016 = yearly_data(df=titans_df2016, year=2016, 
                        HC='Mike Mularkey',
                        OC='Terry Robiskie', 
                        DC="Dick LeBeau", 
                        QB='Marcus Mariota', 
                        QB2='Matt Cassel',QB2_range=17)

titans_df2017 = yearly_data(df=titans_df2017, year=2017, 
                        HC='Mike Mularkey',
                        OC='Terry Robiskie', 
                        DC="Dick LeBeau", 
                        QB='Marcus Mariota', 
                        QB2='Matt Cassel',QB2_range=5)

titans_df2018 = yearly_data(df=titans_df2018, year=2018, 
                        HC='Mike Vrabel', 
                        OC='Matt LaFleur', 
                        DC='Dean Pees', 
                        QB='Marcus Mariota', 
                        QB2='Blaine Gabbert',QB2_range=range(2,4,1))
titans_df2018.loc[titans_df2018['Week'] == 17, 'QB'] = "Blaine Gabbert"

titans_df2019 = yearly_data(df=titans_df2019, year=2019, 
                        HC='Mike Vrabel', 
                        OC='Arthur Smith', 
                        DC='Dean Pees', 
                        QB='Ryan Tannehill', 
                        QB2='Marcus Mariota',QB2_range=range(1,7,1))

# Make master dataframe for team
titans_df = titans_df2019.append(titans_df2018, ignore_index=True)
titans_df = titans_df.append(titans_df2017, ignore_index=True)
titans_df = titans_df.append(titans_df2016, ignore_index=True)
titans_df = titans_df.append(titans_df2015, ignore_index=True)
titans_df = titans_df.append(titans_df2014, ignore_index=True)
titans_df = titans_df.append(titans_df2013, ignore_index=True)
titans_df = titans_df.append(titans_df2012, ignore_index=True)
titans_df = titans_df.append(titans_df2011, ignore_index=True)
titans_df = titans_df.append(titans_df2010, ignore_index=True)

titans_df['Base_Team'] = 'Titans'
titans_df = titans_df.sort_values(by=['year', 'Week'], ascending=[False, True])

titans_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Cleveland Browns,2019,Titans,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Mike Vrabel,Arthur Smith,Dean Pees,Ryan Tannehill
1,2,Sun,0.0,1.0,1.0,Indianapolis Colts,2019,Titans,43.0,13.0,...,3.0,244.0,102.0,339.0,346.0,0,Mike Vrabel,Arthur Smith,Dean Pees,Ryan Tannehill
2,3,Thu,0.0,1.0,0.0,Jacksonville Jaguars,2019,Titans,60.0,32.0,...,5.0,365.0,269.0,582.0,634.0,0,Mike Vrabel,Arthur Smith,Dean Pees,Ryan Tannehill
3,4,Sun,1.0,1.0,0.0,Atlanta Falcons,2019,Titans,67.0,52.0,...,5.0,569.0,357.0,922.0,926.0,0,Mike Vrabel,Arthur Smith,Dean Pees,Ryan Tannehill
4,5,Sun,0.0,2.0,1.0,Buffalo Bills,2019,Titans,91.0,62.0,...,6.0,933.0,415.0,1287.0,1348.0,0,Mike Vrabel,Arthur Smith,Dean Pees,Ryan Tannehill
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,13,Sun,0.0,5.0,1.0,Jacksonville Jaguars,2010,Titans,257.0,218.0,...,21.0,2745.0,1223.0,3279.0,3968.0,0,Jeff Fisher,Mike Heimerdinger,Chuck Cecil,Vince Young
156,14,Thu,0.0,5.0,1.0,Indianapolis Colts,2010,Titans,263.0,235.0,...,21.0,2864.0,1481.0,3499.0,4345.0,0,Jeff Fisher,Mike Heimerdinger,Chuck Cecil,Vince Young
157,15,Sun,1.0,5.0,1.0,Houston Texans,2010,Titans,291.0,265.0,...,21.0,3176.0,1568.0,3864.0,4744.0,0,Jeff Fisher,Mike Heimerdinger,Chuck Cecil,Vince Young
158,16,Sun,0.0,6.0,0.0,Kansas City Chiefs,2010,Titans,322.0,282.0,...,22.0,3469.0,1598.0,4223.0,5067.0,0,Jeff Fisher,Mike Heimerdinger,Chuck Cecil,Vince Young


In [431]:
titans_df.to_csv('Resources/indv_team_csvs/titans.csv')